### grp

# Course: _Introduction to Spark SQL with Python_:
1.  sql
2.  window function
3.  ui
4.  nlp

https://spark.apache.org/docs/latest/api/python/index.html

In [1]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Row
from pyspark.sql import Window

## _1. PySpark SQL_:
-  ```spark.sql(...)```
-  Window Functions:
    -  ```pyspark.sql.Window```
    -  ```pyspark.sql.Window.partitionBy```
    -  ```pyspark.sql.Window.orderBy```
    -  ```pyspark.sql.window.WindowSpec``` => ex: ```window = Window.partitionBy('col1').orderBy('col2')```
    -  aggregate function for every row per group
-  ```agg(...)``` => summarize one column at a time

#### sql table

In [2]:
PATH = "course_data/spark_sql/trainsched.txt"
df = spark.read.csv(PATH, header=True)
df.createOrReplaceTempView("schedule")

In [3]:
spark.table("schedule").show()

+--------+-------------+-----+
|train_id|      station| time|
+--------+-------------+-----+
|     324|San Francisco|7:59a|
|     324|  22nd Street|8:03a|
|     324|     Millbrae|8:16a|
|     324|    Hillsdale|8:24a|
|     324| Redwood City|8:31a|
|     324|    Palo Alto|8:37a|
|     324|     San Jose|9:05a|
|     217|       Gilroy|6:06a|
|     217|   San Martin|6:15a|
|     217|  Morgan Hill|6:21a|
|     217| Blossom Hill|6:36a|
|     217|      Capitol|6:42a|
|     217|       Tamien|6:50a|
|     217|     San Jose|6:59a|
+--------+-------------+-----+



#### table cols

In [4]:
spark.sql("DESCRIBE schedule").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|train_id|   string|   null|
| station|   string|   null|
|    time|   string|   null|
+--------+---------+-------+



#### window function sum

In [5]:
schema = T.StructType([T.StructField('train_id', T.StringType()),\
                       T.StructField('station', T.StringType()),\
                       T.StructField('time', T.StringType()),\
                       T.StructField('diff_min', T.DoubleType())])

In [6]:
df = spark\
.createDataFrame(\
 [Row(train_id='217', station='Gilroy', time='6:06a', diff_min=9.0),
  Row(train_id='217', station='San Martin', time='6:15a', diff_min=6.0),
  Row(train_id='217', station='Morgan Hill', time='6:21a', diff_min=15.0),
  Row(train_id='217', station='Blossom Hill', time='6:36a', diff_min=6.0),
  Row(train_id='217', station='Capitol', time='6:42a', diff_min=8.0),
  Row(train_id='217', station='Tamien', time='6:50a', diff_min=9.0),
  Row(train_id='217', station='San Jose', time='6:59a', diff_min=None),
  Row(train_id='324', station='San Francisco', time='7:59a', diff_min=4.0),
  Row(train_id='324', station='22nd Street', time='8:03a', diff_min=13.0),
  Row(train_id='324', station='Millbrae', time='8:16a', diff_min=8.0),
  Row(train_id='324', station='Hillsdale', time='8:24a', diff_min=7.0),
  Row(train_id='324', station='Redwood City', time='8:31a', diff_min=6.0),
  Row(train_id='324', station='Palo Alto', time='8:37a', diff_min=28.0),
  Row(train_id='324', station='San Jose', time='9:05a', diff_min=None)],\
  schema=schema)

df.createOrReplaceTempView("schedule")

In [7]:
# Add col running_total that sums diff_min col in each group
query = """
SELECT train_id, station, time, diff_min,
SUM(diff_min) OVER (PARTITION BY train_id ORDER BY time) AS running_total
FROM schedule
"""

# Run the query and display the result
spark.sql(query).show()

+--------+-------------+-----+--------+-------------+
|train_id|      station| time|diff_min|running_total|
+--------+-------------+-----+--------+-------------+
|     217|       Gilroy|6:06a|     9.0|          9.0|
|     217|   San Martin|6:15a|     6.0|         15.0|
|     217|  Morgan Hill|6:21a|    15.0|         30.0|
|     217| Blossom Hill|6:36a|     6.0|         36.0|
|     217|      Capitol|6:42a|     8.0|         44.0|
|     217|       Tamien|6:50a|     9.0|         53.0|
|     217|     San Jose|6:59a|    null|         53.0|
|     324|San Francisco|7:59a|     4.0|          4.0|
|     324|  22nd Street|8:03a|    13.0|         17.0|
|     324|     Millbrae|8:16a|     8.0|         25.0|
|     324|    Hillsdale|8:24a|     7.0|         32.0|
|     324| Redwood City|8:31a|     6.0|         38.0|
|     324|    Palo Alto|8:37a|    28.0|         66.0|
|     324|     San Jose|9:05a|    null|         66.0|
+--------+-------------+-----+--------+-------------+



#### partition by => dot notation vs sql notation

In [8]:
bad_query = """
SELECT 
ROW_NUMBER() OVER (ORDER BY time) AS row,
train_id, 
station, 
time, 
LEAD(time,1) OVER (ORDER BY time) AS time_next 
FROM schedule
"""
spark.sql(bad_query).show()

# Give the number of the bad row
bad_row = 7

# Provide the missing clause
clause = "PARTITION BY train_id"

print("="*10)

good_query = """
SELECT 
ROW_NUMBER() OVER (ORDER BY time) AS row,
train_id, 
station, 
time, 
LEAD(time,1) OVER (PARTITION BY train_id ORDER BY time) AS time_next 
FROM schedule
"""
spark.sql(good_query).show()

+---+--------+-------------+-----+---------+
|row|train_id|      station| time|time_next|
+---+--------+-------------+-----+---------+
|  1|     217|       Gilroy|6:06a|    6:15a|
|  2|     217|   San Martin|6:15a|    6:21a|
|  3|     217|  Morgan Hill|6:21a|    6:36a|
|  4|     217| Blossom Hill|6:36a|    6:42a|
|  5|     217|      Capitol|6:42a|    6:50a|
|  6|     217|       Tamien|6:50a|    6:59a|
|  7|     217|     San Jose|6:59a|    7:59a|
|  8|     324|San Francisco|7:59a|    8:03a|
|  9|     324|  22nd Street|8:03a|    8:16a|
| 10|     324|     Millbrae|8:16a|    8:24a|
| 11|     324|    Hillsdale|8:24a|    8:31a|
| 12|     324| Redwood City|8:31a|    8:37a|
| 13|     324|    Palo Alto|8:37a|    9:05a|
| 14|     324|     San Jose|9:05a|     null|
+---+--------+-------------+-----+---------+

+---+--------+-------------+-----+---------+
|row|train_id|      station| time|time_next|
+---+--------+-------------+-----+---------+
|  1|     217|       Gilroy|6:06a|    6:15a|
|  2|    

#### aggregation => dot notation vs sql notation

In [9]:
# Give the identical result in each command
spark.sql('SELECT train_id, MIN(time) AS start FROM schedule GROUP BY train_id').show()
df.groupBy('train_id').agg({'time':'min'}).withColumnRenamed('min(time)', 'start').show()

print("="*10)
print("\n")

# Print the second column of the result
spark.sql('SELECT train_id, MIN(time), MAX(time) FROM schedule GROUP BY train_id').show()
result = df.groupBy('train_id').agg({'time':'min', 'time':'max'})
result.show()
print(result.columns[1])

+--------+-----+
|train_id|start|
+--------+-----+
|     217|6:06a|
|     324|7:59a|
+--------+-----+

+--------+-----+
|train_id|start|
+--------+-----+
|     217|6:06a|
|     324|7:59a|
+--------+-----+



+--------+---------+---------+
|train_id|min(time)|max(time)|
+--------+---------+---------+
|     217|    6:06a|    6:59a|
|     324|    7:59a|    9:05a|
+--------+---------+---------+

+--------+---------+
|train_id|max(time)|
+--------+---------+
|     217|    6:59a|
|     324|    9:05a|
+--------+---------+

max(time)


In [10]:
expr = [F.min(F.col("time")).alias('start'), F.max(F.col("time")).alias('end')]
dot_df = df.groupBy("train_id").agg(*expr)
dot_df.show()

# Write a SQL query giving a result identical to dot_df
query = "SELECT train_id, MIN(time) AS start, MAX(time) AS end FROM schedule GROUP BY train_id"
sql_df = spark.sql(query)
sql_df.show()

+--------+-----+-----+
|train_id|start|  end|
+--------+-----+-----+
|     217|6:06a|6:59a|
|     324|7:59a|9:05a|
+--------+-----+-----+

+--------+-----+-----+
|train_id|start|  end|
+--------+-----+-----+
|     217|6:06a|6:59a|
|     324|7:59a|9:05a|
+--------+-----+-----+



In [11]:
df = spark.sql("""
SELECT *, 
LEAD(time,1) OVER(PARTITION BY train_id ORDER BY time) AS time_next 
FROM schedule
""")

df.show()

# Obtain the identical result using dot notation 
dot_df = df.withColumn('time_next', F.lead('time', 1)
        .over(Window.partitionBy('train_id')
        .orderBy('time')))

dot_df.show()

+--------+-------------+-----+--------+---------+
|train_id|      station| time|diff_min|time_next|
+--------+-------------+-----+--------+---------+
|     217|       Gilroy|6:06a|     9.0|    6:15a|
|     217|   San Martin|6:15a|     6.0|    6:21a|
|     217|  Morgan Hill|6:21a|    15.0|    6:36a|
|     217| Blossom Hill|6:36a|     6.0|    6:42a|
|     217|      Capitol|6:42a|     8.0|    6:50a|
|     217|       Tamien|6:50a|     9.0|    6:59a|
|     217|     San Jose|6:59a|    null|     null|
|     324|San Francisco|7:59a|     4.0|    8:03a|
|     324|  22nd Street|8:03a|    13.0|    8:16a|
|     324|     Millbrae|8:16a|     8.0|    8:24a|
|     324|    Hillsdale|8:24a|     7.0|    8:31a|
|     324| Redwood City|8:31a|     6.0|    8:37a|
|     324|    Palo Alto|8:37a|    28.0|    9:05a|
|     324|     San Jose|9:05a|    null|     null|
+--------+-------------+-----+--------+---------+

+--------+-------------+-----+--------+---------+
|train_id|      station| time|diff_min|time_next|

#### window function => dot notation vs sql notation

In [12]:
df = df.drop("diff_min", "time_next")
df.createOrReplaceTempView("schedule")

In [13]:
window = Window.partitionBy('train_id').orderBy('time')
dot_df = df.withColumn('diff_min', 
                    (F.unix_timestamp(F.lead('time', 1).over(window),'H:m') 
                     - F.unix_timestamp('time', 'H:m'))/60)

dot_df.show()

# Create a SQL query to obtain an identical result to dot_df 
query = """
SELECT *, 
(UNIX_TIMESTAMP(LEAD(time, 1) OVER (PARTITION BY train_id ORDER BY time),'H:m') 
 - UNIX_TIMESTAMP(time, 'H:m'))/60 AS diff_min 
FROM schedule 
"""
sql_df = spark.sql(query)
sql_df.show()

+--------+-------------+-----+--------+
|train_id|      station| time|diff_min|
+--------+-------------+-----+--------+
|     217|       Gilroy|6:06a|     9.0|
|     217|   San Martin|6:15a|     6.0|
|     217|  Morgan Hill|6:21a|    15.0|
|     217| Blossom Hill|6:36a|     6.0|
|     217|      Capitol|6:42a|     8.0|
|     217|       Tamien|6:50a|     9.0|
|     217|     San Jose|6:59a|    null|
|     324|San Francisco|7:59a|     4.0|
|     324|  22nd Street|8:03a|    13.0|
|     324|     Millbrae|8:16a|     8.0|
|     324|    Hillsdale|8:24a|     7.0|
|     324| Redwood City|8:31a|     6.0|
|     324|    Palo Alto|8:37a|    28.0|
|     324|     San Jose|9:05a|    null|
+--------+-------------+-----+--------+

+--------+-------------+-----+--------+
|train_id|      station| time|diff_min|
+--------+-------------+-----+--------+
|     217|       Gilroy|6:06a|     9.0|
|     217|   San Martin|6:15a|     6.0|
|     217|  Morgan Hill|6:21a|    15.0|
|     217| Blossom Hill|6:36a|     6.0|

## _2. Using Window Function SQL for NLP_:
-  ```split(...)```
-  ```explode(...)```
-  ```lag(...)``` => Window function: returns the value that is offset rows before the current row
-  ```lead(...)``` => Window function: returns the value that is offset rows after the current row
-  ```length(...)```

#### read parquet

In [14]:
# Load the dataframe
df = spark.read.load('course_data/spark_sql/sherlock.parquet')

# Filter and show the first 5 rows
df.where('id > 70').show(5, truncate=False)

+------+---+
|word  |id |
+------+---+
|it    |71 |
|do    |72 |
|not   |73 |
|change|74 |
|or    |75 |
+------+---+
only showing top 5 rows



#### split / explode text

In [15]:
schema = T.StructType([T.StructField('clause', T.StringType()),\
                       T.StructField('id', T.LongType())])

In [16]:
df_clauses = spark\
.createDataFrame(\
[Row(clause='title', id=0),
 Row(clause='the adventures of sherlock holmes author', id=1),
 Row(clause='sir arthur conan doyle release date', id=2),
 Row(clause='march 1999', id=3),
 Row(clause='ebook 1661', id=4),
 Row(clause='most recently updated', id=5),
 Row(clause='november 29 2002', id=6),
 Row(clause='edition', id=7),
 Row(clause='12 language', id=8),
 Row(clause='english character set encoding', id=9),
 Row(clause='ascii', id=10),
 Row(clause='start of the project gutenberg ebook the adventures of sherlock holmes', id=11),
 Row(clause='additional editing by jose menendez', id=12),
 Row(clause='the adventures of sherlock holmes by sir arthur conan doyle contents i', id=13),
 Row(clause='a scandal in bohemia ii', id=14),
 Row(clause='the red', id=15),
 Row(clause='headed league iii', id=16),
 Row(clause='a case of identity iv', id=17),
 Row(clause='the boscombe valley mystery v', id=18),
 Row(clause='the five orange pips vi', id=19),
 Row(clause='the man with the twisted lip vii', id=20),
 Row(clause='the adventure of the blue carbuncle viii', id=21),
 Row(clause='the adventure of the speckled band ix', id=22),
 Row(clause='the adventure of the engineer s thumb x', id=23),
 Row(clause='the adventure of the noble bachelor xi', id=24),
 Row(clause='the adventure of the beryl coronet xii', id=25),
 Row(clause='the adventure of the copper beeches adventure i', id=26),
 Row(clause='a scandal in bohemia i', id=27),
 Row(clause='to sherlock holmes she is always the woman', id=28),
 Row(clause='i have seldom heard him mention her under any other name', id=29),
 Row(clause='in his eyes she eclipses and predominates the whole of her sex', id=30),
 Row(clause='it was not that he felt any emotion akin to love for irene adler', id=31),
 Row(clause='all emotions and that one particularly were abhorrent to his cold precise but admirably balanced mind', id=32),
 Row(clause='he was i take it the most perfect reasoning and observing machine that the world has seen but as a lover he would have placed himself in a false position', id=33),
 Row(clause='he never spoke of the softer passions save with a gibe and a sneer', id=34),
 Row(clause='they were admirable things for the observer', id=35),
 Row(clause='excellent for drawing the veil from men s motives and actions', id=36),
 Row(clause='but for the trained reasoner to admit such intrusions into his own delicate and finely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results', id=37),
 Row(clause='grit in a sensitive instrument or a crack in one of his own high', id=38),
 Row(clause='power lenses would not be more disturbing than a strong emotion in a nature such as his', id=39),
 Row(clause='and yet there was but one woman to him and that woman was the late irene adler of dubious and questionable memory', id=40),
 Row(clause='i had seen little of holmes lately', id=41),
 Row(clause='my marriage had drifted us away from each other', id=42),
 Row(clause='my own complete happiness and the home', id=43),
 Row(clause='centred interests which rise up around the man who first finds himself master of his own establishment were sufficient to absorb all my attention while holmes who loathed every form of society with his whole bohemian soul remained in our lodgings in baker street buried among his old books and alternating from week to week between cocaine and ambition the drowsiness of the drug and the fierce energy of his own keen nature', id=44),
 Row(clause='he was still as ever deeply attracted by the study of crime and occupied his immense faculties and extraordinary powers of observation in following out those clues and clearing up those mysteries which had been abandoned as hopeless by the official police', id=45),
 Row(clause='from time to time i heard some vague account of his doings', id=46),
 Row(clause='of his summons to odessa in the case of the trepoff murder of his clearing up of the singular tragedy of the atkinson brothers at trincomalee and finally of the mission which he had accomplished so delicately and successfully for the reigning family of holland', id=47),
 Row(clause='beyond these signs of his activity however which i merely shared with all the readers of the daily press i knew little of my former friend and companion', id=48),
 Row(clause='one night', id=49),
 Row(clause='it was on the twentieth of march 1888', id=50),
 Row(clause='i was returning from a journey to a patient', id=51),
 Row(clause='for i had now returned to civil practice', id=52),
 Row(clause='when my way led me through baker street', id=53),
 Row(clause='as i passed the well', id=54),
 Row(clause='remembered door which must always be associated in my mind with my wooing and with the dark incidents of the study in scarlet i was seized with a keen desire to see holmes again and to know how he was employing his extraordinary powers', id=55),
 Row(clause='his rooms were brilliantly lit and even as i looked up i saw his tall spare figure pass twice in a dark silhouette against the blind', id=56),
 Row(clause='he was pacing the room swiftly eagerly with his head sunk upon his chest and his hands clasped behind him', id=57),
 Row(clause='to me who knew his every mood and habit his attitude and manner told their own story', id=58),
 Row(clause='he was at work again', id=59),
 Row(clause='he had risen out of his drug', id=60),
 Row(clause='created dreams and was hot upon the scent of some new problem', id=61),
 Row(clause='i rang the bell and was shown up to the chamber which had formerly been in part my own', id=62),
 Row(clause='his manner was not effusive', id=63),
 Row(clause='it seldom was', id=64),
 Row(clause='but he was glad i think to see me', id=65),
 Row(clause='with hardly a word spoken but with a kindly eye he waved me to an armchair threw across his case of cigars and indicated a spirit case and a gasogene in the corner', id=66),
 Row(clause='then he stood before the fire and looked me over in his singular introspective fashion', id=67),
 Row(clause='wedlock suits you he remarked', id=68),
 Row(clause='i think watson that you have put on seven and a half pounds since i saw you', id=69),
 Row(clause='seven', id=70),
 Row(clause='i answered', id=71),
 Row(clause='indeed i should have thought a little more', id=72),
 Row(clause='just a trifle more i fancy watson', id=73),
 Row(clause='and in practice again i observe', id=74),
 Row(clause='you did not tell me that you intended to go into harness', id=75),
 Row(clause='then how do you know', id=76),
 Row(clause='i see it i deduce it', id=77),
 Row(clause='how do i know that you have been getting yourself very wet lately and that you have a most clumsy and careless servant girl', id=78),
 Row(clause='my dear holmes said i this is too much', id=79),
 Row(clause='you would certainly have been burned had you lived a few centuries ago', id=80),
 Row(clause='it is true that i had a country walk on thursday and came home in a dreadful mess but as i have changed my clothes i canot imagine how you deduce it', id=81),
 Row(clause='as to mary jane she is incorrigible and my wife has given her notice but there again i fail to see how you work it out', id=82),
 Row(clause='he chuckled to himself and rubbed his long nervous hands together', id=83),
 Row(clause='it is simplicity itself said he', id=84),
 Row(clause='my eyes tell me that on the inside of your left shoe just where the firelight strikes it the leather is scored by six almost parallel cuts', id=85),
 Row(clause='obviously they have been caused by someone who has very carelessly scraped round the edges of the sole in order to remove crusted mud from it', id=86),
 Row(clause='hence you see my double deduction that you had been out in vile weather and that you had a particularly malignant boot', id=87),
 Row(clause='slitting specimen of the london slavey', id=88),
 Row(clause='as to your practice if a gentleman walks into my rooms smelling of iodoform with a black mark of nitrate of silver upon his right forefinger and a bulge on the right side of his top', id=89),
 Row(clause='hat to show where he has secreted his stethoscope i must be dull indeed if i do not pronounce him to be an active member of the medical profession', id=90),
 Row(clause='i could not help laughing at the ease with which he explained his process of deduction', id=91),
 Row(clause='when i hear you give your reasons i remarked the thing always appears to me to be so ridiculously simple that i could easily do it myself though at each successive instance of your reasoning i am baffled until you explain your process', id=92),
 Row(clause='and yet i believe that my eyes are as good as yours', id=93),
 Row(clause='quite so he answered lighting a cigarette and throwing himself down into an armchair', id=94),
 Row(clause='you see but you do not observe', id=95),
 Row(clause='the distinction is clear', id=96),
 Row(clause='for example you have frequently seen the steps which lead up from the hall to this room', id=97),
 Row(clause='frequently', id=98),
 Row(clause='how often', id=99)], schema=schema)

In [17]:
# Split the clause column into a column called words 
split_df = df_clauses.select(F.split('clause',' ').alias('words'))
split_df.show(5, truncate=False)

# Explode the words column into a column called word 
exploded_df = split_df.select(F.explode('words').alias('word'))
exploded_df.show(10)

# Count the resulting number of rows in exploded_df
print("\nNumber of rows: ", exploded_df.count())

+-----------------------------------------------+
|words                                          |
+-----------------------------------------------+
|[title]                                        |
|[the, adventures, of, sherlock, holmes, author]|
|[sir, arthur, conan, doyle, release, date]     |
|[march, 1999]                                  |
|[ebook, 1661]                                  |
+-----------------------------------------------+
only showing top 5 rows

+----------+
|      word|
+----------+
|     title|
|       the|
|adventures|
|        of|
|  sherlock|
|    holmes|
|    author|
|       sir|
|    arthur|
|     conan|
+----------+
only showing top 10 rows


Number of rows:  1279


#### sliding windows

In [18]:
schema = T.StructType([T.StructField('word', T.StringType()),\
                       T.StructField('id', T.LongType()),\
                       T.StructField('part', T.IntegerType()),\
                       T.StructField('title', T.StringType())])

In [19]:
df_text = spark\
.createDataFrame(\
[Row(word='scandal', id=305, part=1, title='Sherlock Chapter I'),
 Row(word='in', id=306, part=1, title='Sherlock Chapter I'),
 Row(word='bohemia', id=307, part=1, title='Sherlock Chapter I'),
 Row(word='i', id=308, part=1, title='Sherlock Chapter I'),
 Row(word='to', id=309, part=1, title='Sherlock Chapter I'),
 Row(word='sherlock', id=310, part=1, title='Sherlock Chapter I'),
 Row(word='holmes', id=311, part=1, title='Sherlock Chapter I'),
 Row(word='she', id=312, part=1, title='Sherlock Chapter I'),
 Row(word='is', id=313, part=1, title='Sherlock Chapter I'),
 Row(word='red-headed', id=8861, part=2, title='Sherlock Chapter II'),
 Row(word='league', id=8862, part=2, title='Sherlock Chapter II'),
 Row(word='i', id=8863, part=2, title='Sherlock Chapter II'),
 Row(word='had', id=8864, part=2, title='Sherlock Chapter II'),
 Row(word='called', id=8865, part=2, title='Sherlock Chapter II'),
 Row(word='upon', id=8866, part=2, title='Sherlock Chapter II'),
 Row(word='my', id=8867, part=2, title='Sherlock Chapter II'),
 Row(word='friend', id=8868, part=2, title='Sherlock Chapter II'),
 Row(word='mr', id=8869, part=2, title='Sherlock Chapter II'),
 Row(word='sherlock', id=8870, part=2, title='Sherlock Chapter II'),
 Row(word='case', id=18032, part=3, title='Sherlock Chapter III'),
 Row(word='of', id=18033, part=3, title='Sherlock Chapter III'),
 Row(word='identity', id=18034, part=3, title='Sherlock Chapter III'),
 Row(word='my', id=18035, part=3, title='Sherlock Chapter III'),
 Row(word='dear', id=18036, part=3, title='Sherlock Chapter III'),
 Row(word='fellow', id=18037, part=3, title='Sherlock Chapter III'),
 Row(word='said', id=18038, part=3, title='Sherlock Chapter III'),
 Row(word='sherlock', id=18039, part=3, title='Sherlock Chapter III'),
 Row(word='holmes', id=18040, part=3, title='Sherlock Chapter III'),
 Row(word='as', id=18041, part=3, title='Sherlock Chapter III'),
 Row(word='always', id=314, part=1, title='Sherlock Chapter I'),
 Row(word='ix', id=68993, part=9, title='Sherlock Chapter IX'),
 Row(word='the', id=68994, part=9, title='Sherlock Chapter IX'),
 Row(word='adventure', id=68995, part=9, title='Sherlock Chapter IX'),
 Row(word='of', id=68996, part=9, title='Sherlock Chapter IX'),
 Row(word='the', id=68997, part=9, title='Sherlock Chapter IX'),
 Row(word='engineer', id=68998, part=9, title='Sherlock Chapter IX'),
 Row(word='s', id=68999, part=9, title='Sherlock Chapter IX'),
 Row(word='thumb', id=69000, part=9, title='Sherlock Chapter IX'),
 Row(word='of', id=69001, part=9, title='Sherlock Chapter IX'),
 Row(word='all', id=69002, part=9, title='Sherlock Chapter IX'),
 Row(word='x', id=77313, part=10, title='Sherlock Chapter X'),
 Row(word='the', id=77314, part=10, title='Sherlock Chapter X'),
 Row(word='adventure', id=77315, part=10, title='Sherlock Chapter X'),
 Row(word='of', id=77316, part=10, title='Sherlock Chapter X'),
 Row(word='the', id=77317, part=10, title='Sherlock Chapter X'),
 Row(word='noble', id=77318, part=10, title='Sherlock Chapter X'),
 Row(word='bachelor', id=77319, part=10, title='Sherlock Chapter X'),
 Row(word='the', id=77320, part=10, title='Sherlock Chapter X'),
 Row(word='lord', id=77321, part=10, title='Sherlock Chapter X'),
 Row(word='st', id=77322, part=10, title='Sherlock Chapter X'),
 Row(word='xi', id=85462, part=11, title='Sherlock Chapter XI'),
 Row(word='the', id=85463, part=11, title='Sherlock Chapter XI'),
 Row(word='adventure', id=85464, part=11, title='Sherlock Chapter XI'),
 Row(word='of', id=85465, part=11, title='Sherlock Chapter XI'),
 Row(word='the', id=85466, part=11, title='Sherlock Chapter XI'),
 Row(word='beryl', id=85467, part=11, title='Sherlock Chapter XI'),
 Row(word='coronet', id=85468, part=11, title='Sherlock Chapter XI'),
 Row(word='holmes', id=85469, part=11, title='Sherlock Chapter XI'),
 Row(word='said', id=85470, part=11, title='Sherlock Chapter XI'),
 Row(word='i', id=85471, part=11, title='Sherlock Chapter XI'),
 Row(word='xii', id=95166, part=12, title='Sherlock Chapter XII'),
 Row(word='the', id=95167, part=12, title='Sherlock Chapter XII'),
 Row(word='adventure', id=95168, part=12, title='Sherlock Chapter XII'),
 Row(word='of', id=95169, part=12, title='Sherlock Chapter XII'),
 Row(word='the', id=95170, part=12, title='Sherlock Chapter XII'),
 Row(word='copper', id=95171, part=12, title='Sherlock Chapter XII'),
 Row(word='beeches', id=95172, part=12, title='Sherlock Chapter XII'),
 Row(word='to', id=95173, part=12, title='Sherlock Chapter XII'),
 Row(word='the', id=95174, part=12, title='Sherlock Chapter XII'),
 Row(word='man', id=95175, part=12, title='Sherlock Chapter XII')], schema=schema)

In [20]:
df_text.createOrReplaceTempView("text")

In [21]:
query = """
SELECT
part,
LAG(word, 2) OVER(PARTITION BY part ORDER BY id) AS w1,
LAG(word, 1) OVER(PARTITION BY part ORDER BY id) AS w2,
word AS w3,
LEAD(word, 1) OVER(PARTITION BY part ORDER BY id) AS w4,
LEAD(word, 2) OVER(PARTITION BY part ORDER BY id) AS w5
FROM text
"""
spark.sql(query).where("part = 12").show(10)

+----+---------+---------+---------+---------+---------+
|part|       w1|       w2|       w3|       w4|       w5|
+----+---------+---------+---------+---------+---------+
|  12|     null|     null|      xii|      the|adventure|
|  12|     null|      xii|      the|adventure|       of|
|  12|      xii|      the|adventure|       of|      the|
|  12|      the|adventure|       of|      the|   copper|
|  12|adventure|       of|      the|   copper|  beeches|
|  12|       of|      the|   copper|  beeches|       to|
|  12|      the|   copper|  beeches|       to|      the|
|  12|   copper|  beeches|       to|      the|      man|
|  12|  beeches|       to|      the|      man|     null|
|  12|       to|      the|      man|     null|     null|
+----+---------+---------+---------+---------+---------+



#### repartition

In [22]:
df_text.select('title')\
       .distinct()\
       .orderBy('title')\
       .show(truncate=False)

+--------------------+
|title               |
+--------------------+
|Sherlock Chapter I  |
|Sherlock Chapter II |
|Sherlock Chapter III|
|Sherlock Chapter IX |
|Sherlock Chapter X  |
|Sherlock Chapter XI |
|Sherlock Chapter XII|
+--------------------+



In [23]:
df_text_part = df_text.coalesce(1) # make 1 partition for testing purposes

In [24]:
df_text_part.rdd.getNumPartitions() # check # of partitions

1

The dataframe ```text_df_part``` is currently in a single partition. 
Suppose that you know that the upcoming processing steps are going to be grouping the data on titles. 
Processing the data will be most efficient if each title stays within a single machine. 
To avoid unnecessary shuffling of the data from one machine to another, let's repartition the dataframe into one partition per title, using the ```repartition``` and ```getNumPartitions``` commands.

In [25]:
# Repartition text_df into 7 partitions on 'title' column
df_repart = df_text_part.repartition(7, 'title')

# Prove that repart_df has 7 partitions
df_repart.rdd.getNumPartitions()

7

#### common word sequences =>  3-tuples in the text document

In [26]:
# Find the top 10 sequences of three words
query = """
SELECT w1, w2, w3, COUNT(*) AS count FROM (
   SELECT word AS w1,
   LEAD(word,1) OVER(PARTITION BY part ORDER BY id ) AS w2,
   LEAD(word,2) OVER(PARTITION BY part ORDER BY id ) AS w3
FROM text
)
GROUP BY w1, w2, w3
ORDER BY count DESC
LIMIT 10
""" 
df = spark.sql(query)
df.show()

+----------+---------+------+-----+
|        w1|       w2|    w3|count|
+----------+---------+------+-----+
| adventure|       of|   the|    4|
|       the|adventure|    of|    4|
|       the|     lord|    st|    1|
|       the|      man|  null|    1|
|  bachelor|      the|  lord|    1|
|red-headed|   league|     i|    1|
|    always|     null|  null|    1|
|       she|       is|always|    1|
|    called|     upon|    my|    1|
|      said| sherlock|holmes|    1|
+----------+---------+------+-----+



#### common word sequences =>  sorted alpha desc order

In [27]:
# Unique 3-tuples sorted in descending order
spark.sql("""
SELECT DISTINCT w1, w2, w3 FROM (
   SELECT word AS w1,
   LEAD(word,1) OVER(PARTITION BY part ORDER BY id ) AS w2,
   LEAD(word,2) OVER(PARTITION BY part ORDER BY id ) AS w3
   FROM text
)
ORDER BY w1 DESC, w2 DESC, w3 DESC
LIMIT 10
""").show()

+-----+--------+---------+
|   w1|      w2|       w3|
+-----+--------+---------+
|  xii|     the|adventure|
|   xi|     the|adventure|
|    x|     the|adventure|
| upon|      my|   friend|
|   to|     the|      man|
|   to|sherlock|   holmes|
|thumb|      of|      all|
|  the|   noble| bachelor|
|  the|     man|     null|
|  the|    lord|       st|
+-----+--------+---------+



#### common word sequences =>  frequent 3-tuples per chapter

In [28]:
subquery = """
SELECT title, w1, w2, w3, COUNT(*) as count
FROM
(
    SELECT
    title,
    word AS w1,
    LEAD(word, 1) OVER(PARTITION BY title ORDER BY id ) AS w2,
    LEAD(word, 2) OVER(PARTITION BY title ORDER BY id ) AS w3
    FROM text
)
GROUP BY title, w1, w2, w3
ORDER BY title, count DESC
"""

In [29]:
spark.sql(subquery).show()

+-------------------+----------+--------+--------+-----+
|              title|        w1|      w2|      w3|count|
+-------------------+----------+--------+--------+-----+
| Sherlock Chapter I|    always|    null|    null|    1|
| Sherlock Chapter I|   bohemia|       i|      to|    1|
| Sherlock Chapter I|  sherlock|  holmes|     she|    1|
| Sherlock Chapter I|        is|  always|    null|    1|
| Sherlock Chapter I|       she|      is|  always|    1|
| Sherlock Chapter I|    holmes|     she|      is|    1|
| Sherlock Chapter I|   scandal|      in| bohemia|    1|
| Sherlock Chapter I|        to|sherlock|  holmes|    1|
| Sherlock Chapter I|         i|      to|sherlock|    1|
| Sherlock Chapter I|        in| bohemia|       i|    1|
|Sherlock Chapter II|      upon|      my|  friend|    1|
|Sherlock Chapter II|        mr|sherlock|    null|    1|
|Sherlock Chapter II|       had|  called|    upon|    1|
|Sherlock Chapter II|    called|    upon|      my|    1|
|Sherlock Chapter II|        my

In [30]:
#   Most frequent 3-tuple per chapter
query = """
SELECT title, w1, w2, w3, count FROM
(
  SELECT
  title,
  ROW_NUMBER() OVER (PARTITION BY title ORDER BY count DESC) AS row,
  w1, w2, w3, count
  FROM ( %s )
)
WHERE row = 1
ORDER BY title ASC
""" % subquery

spark.sql(query).show()

+--------------------+----------+------+---------+-----+
|               title|        w1|    w2|       w3|count|
+--------------------+----------+------+---------+-----+
|  Sherlock Chapter I|   scandal|    in|  bohemia|    1|
| Sherlock Chapter II|red-headed|league|        i|    1|
|Sherlock Chapter III|      case|    of| identity|    1|
| Sherlock Chapter IX|        ix|   the|adventure|    1|
|  Sherlock Chapter X|         x|   the|adventure|    1|
| Sherlock Chapter XI|        xi|   the|adventure|    1|
|Sherlock Chapter XII|       xii|   the|adventure|    1|
+--------------------+----------+------+---------+-----+



## _3. Caching, Logging, and the Spark UI_:
-  Caching:
    -  keeps data in memory
    -  lazy operation
    -  only cachne if more than one operation needs to be performed on object
    -  unpersist when object is no longer needed
    -  ```cache()``` => cache DF
    -  ```persist()``` => caches DF with ability customizing storage level
    -  ```is_cached```
    -  ```storageLevel``` ... DF defaults:
        -  useDisk = True
        -  useMemory = True
        -  useOffHeap = False
        -  deserialized = True
        -  replication = 1
    -  ```unpersist()``` => uncache DF
    -  ```spark.catalog.cacheTable('...')```
    -  ```spark.catalog.isCached(tableName = '...')```
    -  ```spark.catalog.uncacheTable('...')```
    -  ```spark.catalog.clearCache()```
-  Spark UI:
    -  runs on the driver host
    -  ```Spark Task``` => unit of execution aka **partition** running on a single cpu core aka **thread slot**
    -  ```Spark Stage``` => group of tasks that perform same computation in parallel
    -  ```Spark Job``` => computation trigged by **action** sliced into one or more **stages**
-  Logging
-  Query Plan

#### caching

In [31]:
import time
def prep(df1, df2):
    global begin
    df1.unpersist()
    df2.unpersist()
    begin = time.time()

def print_elapsed():
    print("Overall elapsed : %.1f" % (time.time() - begin))

def run(df, name, elapsed=False):
    start=time.time()
    df.count()
    print("%s : %.1fs" % (name, (time.time()-start)))
    if elapsed:
        print_elapsed()

In [32]:
df1 = spark.range(1000000)
df2 = spark.range(1000000)

# Unpersists df1 and df2 and initializes a timer
prep(df1, df2) 

# Cache df1
df1.cache()

# Run actions on both dataframes
run(df1, "df1_1st") 
run(df1, "df1_2nd")
run(df2, "df2_1st")
run(df2, "df2_2nd", elapsed=True)

# Prove df1 is cached
print(df1.is_cached)

df1_1st : 0.7s
df1_2nd : 0.1s
df2_1st : 0.1s
df2_2nd : 0.0s
Overall elapsed : 0.8
True


In [33]:
from pyspark import StorageLevel

# Unpersist df1 and df2 and initializes a timer
prep(df1, df2) 

# Persist df2 using memory and disk storage level 
df2.persist(storageLevel=StorageLevel(True, True, False, False, 1)) # MEMORY_AND_DISK

# Run actions both dataframes
run(df1, "df1_1st") 
run(df1, "df1_2nd")
run(df2, "df2_1st")
run(df2, "df2_2nd", elapsed=True)

df1_1st : 0.3s
df1_2nd : 0.1s
df2_1st : 0.0s
df2_2nd : 0.0s
Overall elapsed : 0.4


#### uncaching

In [34]:
# List the tables
print("Tables:\n", spark.catalog.listTables())

# Cache table1 and Confirm that it is cached
spark.catalog.cacheTable('text')
print("table1 is cached: ", spark.catalog.isCached('text'))

# Uncache table1 and confirm that it is uncached
spark.catalog.uncacheTable('text')
print("table1 is cached: ", spark.catalog.isCached('text'))

Tables:
 [Table(name='schedule', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='text', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
table1 is cached:  True
table1 is cached:  False


#### logging:
-  DEBUG - debug message
-  INFO - info message
-  WARNING - warn message
-  ERROR - error message

In [35]:
#import logging
#import sys
#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
#                    format='%(levelname)s - %(message)s')

In [36]:
'''
# Log columns of text_df as debug message
logging.debug("text_df columns: %s", text_df.columns)

# Log whether table1 is cached as info message
logging.info("table1 is cached: %s", spark.catalog.isCached(tableName="table1"))

# Log first row of text_df as warning message
logging.warning("The first row of text_df:\n %s", text_df.first())

# Log selected columns of text_df as error message
logging.error("Selected columns: %s", text_df.select("id", "word"))
'''


'\n# Log columns of text_df as debug message\nlogging.debug("text_df columns: %s", text_df.columns)\n\n# Log whether table1 is cached as info message\nlogging.info("table1 is cached: %s", spark.catalog.isCached(tableName="table1"))\n\n# Log first row of text_df as warning message\nlogging.warning("The first row of text_df:\n %s", text_df.first())\n\n# Log selected columns of text_df as error message\nlogging.error("Selected columns: %s", text_df.select("id", "word"))\n'

In [37]:
'''
# Uncomment the statements that do NOT trigger text_df
logging.debug("text_df columns: %s", text_df.columns)
logging.info("table1 is cached: %s", spark.catalog.isCached(tableName="table1"))
# logging.warning("The first row of text_df: %s", text_df.first())
logging.error("Selected columns: %s", text_df.select("id", "word"))
logging.info("Tables: %s", spark.sql("SHOW tables").collect())
logging.debug("First row: %s", spark.sql("SELECT * FROM table1 LIMIT 1"))
# logging.debug("Count: %s", spark.sql("SELECT COUNT(*) AS count FROM table1").collect())
'''

'\n# Uncomment the statements that do NOT trigger text_df\nlogging.debug("text_df columns: %s", text_df.columns)\nlogging.info("table1 is cached: %s", spark.catalog.isCached(tableName="table1"))\n# logging.warning("The first row of text_df: %s", text_df.first())\nlogging.error("Selected columns: %s", text_df.select("id", "word"))\nlogging.info("Tables: %s", spark.sql("SHOW tables").collect())\nlogging.debug("First row: %s", spark.sql("SELECT * FROM table1 LIMIT 1"))\n# logging.debug("Count: %s", spark.sql("SELECT COUNT(*) AS count FROM table1").collect())\n'

#### explain plan

In [38]:
# Run explain on text_df
df_text.explain()

print("="*100)

# Run explain on "SELECT COUNT(*) AS count" query on table1
spark.sql("SELECT COUNT(*) AS count FROM text").explain()

print("="*100)

# Run explain on "COUNT(DISTINCT word) AS words"
spark.sql("SELECT COUNT(DISTINCT word) AS words FROM text").explain()

== Physical Plan ==
Scan ExistingRDD[word#563,id#564L,part#565,title#566]
== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[count(1)])
+- Exchange SinglePartition
   +- *(1) HashAggregate(keys=[], functions=[partial_count(1)])
      +- *(1) Project
         +- Scan ExistingRDD[word#563,id#564L,part#565,title#566]
== Physical Plan ==
*(3) HashAggregate(keys=[], functions=[count(distinct word#563)])
+- Exchange SinglePartition
   +- *(2) HashAggregate(keys=[], functions=[partial_count(distinct word#563)])
      +- *(2) HashAggregate(keys=[word#563], functions=[])
         +- Exchange hashpartitioning(word#563, 200)
            +- *(1) HashAggregate(keys=[word#563], functions=[])
               +- *(1) Project [word#563]
                  +- Scan ExistingRDD[word#563,id#564L,part#565,title#566]


In [39]:
'''

part2_df.explain()

== Physical Plan ==
*(1) Project [word#0, id#1L, part#2, title#3]
+- *(1) Filter (isnotnull(part#2) && (part#2 = 2))
   +- *(1) FileScan parquet [word#0,id#1L,part#2,title#3] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/tmp/tmptef3j2qh/sherlock_parts.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(part), EqualTo(part,2)], ReadSchema: struct<word:string,id:bigint,part:int,title:string>

part3_df.explain()

== Physical Plan ==
InMemoryTableScan [word#9, id#10L, part#11, title#12]
   +- InMemoryRelation [word#9, id#10L, part#11, title#12], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(1) Project [word#9, id#10L, part#11, title#12]
            +- *(1) Filter (isnotnull(part#11) && (part#11 = 4))
               +- *(1) FileScan parquet [word#9,id#10L,part#11,title#12] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/tmp/tmptef3j2qh/sherlock_parts.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(part), EqualTo(part,4)], ReadSchema: struct<word:string,id:bigint,part:int,title:string>
               
part4_df.explain()

== Physical Plan ==
*(1) FileScan parquet [word#38,id#39L] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/tmp/tmptef3j2qh/sherlock.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<word:string,id:bigint>

'''

'\n\npart2_df.explain()\n\n== Physical Plan ==\n*(1) Project [word#0, id#1L, part#2, title#3]\n+- *(1) Filter (isnotnull(part#2) && (part#2 = 2))\n   +- *(1) FileScan parquet [word#0,id#1L,part#2,title#3] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/tmp/tmptef3j2qh/sherlock_parts.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(part), EqualTo(part,2)], ReadSchema: struct<word:string,id:bigint,part:int,title:string>\n\npart3_df.explain()\n\n== Physical Plan ==\nInMemoryTableScan [word#9, id#10L, part#11, title#12]\n   +- InMemoryRelation [word#9, id#10L, part#11, title#12], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)\n         +- *(1) Project [word#9, id#10L, part#11, title#12]\n            +- *(1) Filter (isnotnull(part#11) && (part#11 = 4))\n               +- *(1) FileScan parquet [word#9,id#10L,part#11,title#12] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/tmp/tmptef3j2qh/sherlock_parts.parquet], PartitionFilter

In [40]:
# The filename that part2_df was loaded from
answer1 = 'sherlock_parts.parquet'

# The (integer) part is loaded in part3_df
answer2 = 4

# The filename that part4_df loaded from
answer3 = 'sherlock.parquet'

# The value of the ReadSchema property in part4_df.explain() 
answer4 = 'struct<word:string,id:bigint>'

## _4. Text Classification_:
-  ETS:
    -  Extract
    -  Transform
    -  Selection
-  Train/Test Split
-  Evaluate (Binary Classification):
    -  Probability Vector => [False %, True %]
    -  AUC => classification accuracy

#### udf

In [41]:
schema = T.StructType([T.StructField('doc', T.ArrayType(T.StringType())),\
                       T.StructField('in_', T.ArrayType(T.StringType())),\
                       T.StructField('out', T.ArrayType(T.StringType()))])

In [42]:
df_docs = spark\
.createDataFrame(\
[Row(doc=['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes', 'by', 'sir', 'arthur', 'conan', 'doyle'], in_=['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes', 'by', 'sir', 'arthur', 'conan'], out=['doyle']),
 Row(doc=['15', 'in_', 'our', 'series', 'by', 'sir', 'arthur', 'conan', 'doyle'], in_=['15', 'in_', 'our', 'series', 'by', 'sir', 'arthur', 'conan'], out=['doyle']),
 Row(doc=['copyright', 'laws', 'are', 'changin_g', 'all', 'over', 'the', 'world'], in_=['copyright', 'laws', 'are', 'changin_g', 'all', 'over', 'the'], out=['world']),
 Row(doc=['be', 'sure', 'to', 'check', 'the', 'copyright', 'laws', 'for', 'your', 'country', 'before', 'downloadin_g', 'or', 'redistributin_g', 'this', 'or', 'any', 'other', 'project', 'gutenberg', 'ebook'], in_=['be', 'sure', 'to', 'check', 'the', 'copyright', 'laws', 'for', 'your', 'country', 'before', 'downloadin_g', 'or', 'redistributin_g', 'this', 'or', 'any', 'other', 'project', 'gutenberg'], out=['ebook']),
 Row(doc=['this', 'header', 'should', 'be', 'the', 'first', 'thin_g', 'seen', 'when', 'viewin_g', 'this', 'project', 'gutenberg', 'file'], in_=['this', 'header', 'should', 'be', 'the', 'first', 'thin_g', 'seen', 'when', 'viewin_g', 'this', 'project', 'gutenberg'], out=['file']),
 Row(doc=['please', 'do', 'not', 'remove', 'it'], in_=['please', 'do', 'not', 'remove'], out=['it']),
 Row(doc=['she', 'left', 'this', 'mornin_g', 'with', 'her', 'husband', 'by', 'the', '5'], in_=['by', 'this', 'with', 'left', 'mornin_g', 'the', 'she', 'husband', 'her'], out=['5']),
 Row(doc=['he', 'had', 'no', 'occupation', 'but', 'was', 'in_terested', 'in_', 'several', 'companies', 'and', 'went', 'in_to', 'town', 'as', 'a', 'rule', 'in_', 'the', 'mornin_g', 'returnin_g', 'by', 'the', '5'], in_=['by', 'was', 'companies', 'he', 'in_terested', 'as', 'a', 'returnin_g', 'and', 'occupation', 'but', 'mornin_g', 'in_', 'the', 'rule', 'had', 'several', 'town', 'no', 'went', 'in_to'], out=['5']),
 Row(doc=['5', 'vols'], in_=[], out=['vols']),
 Row(doc=['on', 'the', 'night', 'of', 'march', '5', '1770', 'a', 'crowd', 'on', 'the', 'streets', 'of', 'boston', 'began', 'to', 'jostle', 'and', 'tease', 'some', 'british', 'regulars', 'stationed', 'in_', 'the', 'town'], in_=['stationed', 'crowd', 'on', 'streets', 'night', 'march', 'a', '1770', 'boston', 'to', 'british', 'in_', 'the', 'tease', 'of', 'jostle', 'regulars', 'and', 'began', 'some'], out=['town']),
 Row(doc=['by', 'hurried', 'and', 'irregular', 'methods', 'delegates', 'were', 'elected', 'durin_g', 'the', 'summer', 'and', 'on', 'september', '5', 'the', 'congress', 'duly', 'assembled', 'in_', 'carpenter', 's', 'hall', 'in_', 'philadelphia'], in_=['carpenter', 'by', 'congress', 'were', 'hurried', 'delegates', 'elected', 'september', 'methods', 'irregular', 'in_', 'the', 'on', 'durin_g', 'duly', 'assembled', 'summer', 'and', 'hall'], out=['philadelphia']),
 Row(doc=['do', 'not', 'change', 'or', 'edit', 'the', 'header', 'without', 'written', 'permission'], in_=['do', 'not', 'change', 'or', 'edit', 'the', 'header', 'without', 'written'], out=['permission']),
 Row(doc=['please', 'read', 'the', 'legal', 'small', 'prin_t', 'and', 'other', 'in_formation', 'about', 'the', 'ebook', 'and', 'project', 'gutenberg', 'at', 'the', 'bottom', 'of', 'this', 'file'], in_=['please', 'read', 'the', 'legal', 'small', 'prin_t', 'and', 'other', 'in_formation', 'about', 'the', 'ebook', 'and', 'project', 'gutenberg', 'at', 'the', 'bottom', 'of', 'this'], out=['file']),
 Row(doc=['in_cluded', 'is', 'important', 'in_formation', 'about', 'your', 'specific', 'rights', 'and', 'restrictions', 'in_', 'how', 'the', 'file', 'may', 'be', 'used'], in_=['in_cluded', 'is', 'important', 'in_formation', 'about', 'your', 'specific', 'rights', 'and', 'restrictions', 'in_', 'how', 'the', 'file', 'may', 'be'], out=['used']),
 Row(doc=['you', 'can', 'also', 'fin_d', 'out', 'about', 'how', 'to', 'make', 'a', 'donation', 'to', 'project', 'gutenberg', 'and', 'how', 'to', 'get', 'in_volved'], in_=['you', 'can', 'also', 'fin_d', 'out', 'about', 'how', 'to', 'make', 'a', 'donation', 'to', 'project', 'gutenberg', 'and', 'how', 'to', 'get'], out=['in_volved'])],\
 schema=schema)

In [43]:
df_docs.where(F.array_contains('doc','sherlock')).show()

+--------------------+--------------------+-------+
|                 doc|                 in_|    out|
+--------------------+--------------------+-------+
|[the, project, gu...|[the, project, gu...|[doyle]|
+--------------------+--------------------+-------+



In [44]:
# Returns true if the value is a nonempty vector
nonempty_udf = F.udf(lambda x:  
    True if (x and hasattr(x, "toArray") and x.numNonzeros())
    else False, T.BooleanType())

# Returns first element of the array as string
s_udf = F.udf(lambda x: str(x[0]) if (x and type(x) is list and len(x) > 0)
    else '', T.StringType())

#### array col

In [45]:
TRIVIAL_TOKENS = {'','0','1','2','3','4','5','6','7','8','9','b','c','e',
                  'f','g','h','j','k','l','m','n','o','p','pp','q','r',
                  's','t','u','v','w','x','y','z'}

In [46]:
# Show the rows where doc contains the item '5'
df_docs.where(F.array_contains('doc', '5')).show()

# UDF removes items in TRIVIAL_TOKENS from array
rm_trivial_udf = F.udf(lambda x:
                     list(set(x) - TRIVIAL_TOKENS) if x
                     else x,
                     T.ArrayType(T.StringType()))

# Remove trivial tokens from 'in' and 'out' columns of df2
df_after = df_docs.withColumn('in_', rm_trivial_udf('in_'))\
                    .withColumn('out', rm_trivial_udf('out'))

# Show the rows of df_after where doc contains the item '5'
df_after.where(F.array_contains('doc','5')).show()

+--------------------+--------------------+--------------+
|                 doc|                 in_|           out|
+--------------------+--------------------+--------------+
|[she, left, this,...|[by, this, with, ...|           [5]|
|[he, had, no, occ...|[by, was, compani...|           [5]|
|           [5, vols]|                  []|        [vols]|
|[on, the, night, ...|[stationed, crowd...|        [town]|
|[by, hurried, and...|[carpenter, by, c...|[philadelphia]|
+--------------------+--------------------+--------------+

+--------------------+--------------------+--------------+
|                 doc|                 in_|           out|
+--------------------+--------------------+--------------+
|[she, left, this,...|[by, this, with, ...|            []|
|[he, had, no, occ...|[in_terested, was...|            []|
|           [5, vols]|                  []|        [vols]|
|[on, the, night, ...|[stationed, crowd...|        [town]|
|[by, hurried, and...|[carpenter, by, c...|[philadelphi

#### vector data udf

In [47]:
from pyspark.ml.linalg import SparseVector, VectorUDT

schema = T.StructType([T.StructField('output', VectorUDT())])

In [48]:
df_vector = spark\
.createDataFrame(\
[Row(output=SparseVector(12847, {65: 1.0})),
 Row(output=SparseVector(12847, {8: 1.0})),
 Row(output=SparseVector(12847, {47: 1.0})),
 Row(output=SparseVector(12847, {89: 1.0})),
 Row(output=SparseVector(12847, {94: 1.0})),
 Row(output=SparseVector(12847, {88: 1.0})),
 Row(output=SparseVector(12847, {43: 1.0})),
 Row(output=SparseVector(12847, {164: 1.0})),
 Row(output=SparseVector(12847, {91: 1.0})),
 Row(output=SparseVector(12847, {112: 1.0})),
 Row(output=SparseVector(12847, {97: 1.0})),
 Row(output=SparseVector(12847, {98: 1.0})),
 Row(output=SparseVector(12847, {48: 1.0})),
 Row(output=SparseVector(12847, {66: 1.0})),
 Row(output=SparseVector(12847, {74: 1.0})),
 Row(output=SparseVector(12847, {182: 1.0})),
 Row(output=SparseVector(12847, {118: 1.0})),
 Row(output=SparseVector(12847, {67: 1.0})),
 Row(output=SparseVector(12847, {53: 1.0})),
 Row(output=SparseVector(12847, {85: 1.0})),
 Row(output=SparseVector(12847, {2: 1.0})),
 Row(output=SparseVector(12847, {82: 1.0})),
 Row(output=SparseVector(12847, {29: 1.0})),
 Row(output=SparseVector(12847, {81: 1.0})),
 Row(output=SparseVector(12847, {80: 1.0})),
 Row(output=SparseVector(12847, {145: 1.0})),
 Row(output=SparseVector(12847, {117: 1.0})),
 Row(output=SparseVector(12847, {99: 1.0})),
 Row(output=SparseVector(12847, {72: 1.0})),
 Row(output=SparseVector(12847, {108: 1.0})),
 Row(output=SparseVector(12847, {147: 1.0})),
 Row(output=SparseVector(12847, {139: 1.0})),
 Row(output=SparseVector(12847, {191: 1.0})),
 Row(output=SparseVector(12847, {153: 1.0})),
 Row(output=SparseVector(12847, {175: 1.0})),
 Row(output=SparseVector(12847, {114: 1.0})),
 Row(output=SparseVector(12847, {56: 1.0})),
 Row(output=SparseVector(12847, {86: 1.0})),
 Row(output=SparseVector(12847, {38: 1.0})),
 Row(output=SparseVector(12847, {119: 1.0})),
 Row(output=SparseVector(12847, {95: 1.0})),
 Row(output=SparseVector(12847, {187: 1.0})),
 Row(output=SparseVector(12847, {25: 1.0})),
 Row(output=SparseVector(12847, {11: 1.0})),
 Row(output=SparseVector(12847, {168: 1.0})),
 Row(output=SparseVector(12847, {111: 1.0})),
 Row(output=SparseVector(12847, {123: 1.0})),
 Row(output=SparseVector(12847, {60: 1.0})),
 Row(output=SparseVector(12847, {57: 1.0})),
 Row(output=SparseVector(12847, {36: 1.0})),
 Row(output=SparseVector(12847, {63: 1.0})),
 Row(output=SparseVector(12847, {16: 1.0})),
 Row(output=SparseVector(12847, {110: 1.0})),
 Row(output=SparseVector(12847, {101: 1.0})),
 Row(output=SparseVector(12847, {5: 1.0})),
 Row(output=SparseVector(12847, {78: 1.0}))], schema=schema)

In [49]:
# Selects the first element of a vector column
first_udf = F.udf(lambda x:
            float(x.indices[0]) 
            if (x and hasattr(x, "toArray") and x.numNonzeros())
            else 0.0,
            T.FloatType())

# Use select on df to apply first_udf to the output column
df_vector.select(first_udf("output").alias("result")).show(5)

+------+
|result|
+------+
|  65.0|
|   8.0|
|  47.0|
|  89.0|
|  94.0|
+------+
only showing top 5 rows



In [50]:
# Add label by applying the first_udf to output column
df_new = df_vector.withColumn('label', first_udf('output'))

# Show the first five rows 
df_new.show(5)

+------------------+-----+
|            output|label|
+------------------+-----+
|(12847,[65],[1.0])| 65.0|
| (12847,[8],[1.0])|  8.0|
|(12847,[47],[1.0])| 47.0|
|(12847,[89],[1.0])| 89.0|
|(12847,[94],[1.0])| 94.0|
+------------------+-----+
only showing top 5 rows



#### transform text to vector format

In [51]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="words_in", outputCol="vec")
df_docs = df_docs.withColumnRenamed("in_", "words_in")
model = cv.fit(df_docs)

cv = CountVectorizer(inputCol="words_out", outputCol="vec")
df_docs = df_docs.withColumnRenamed("out", "words_out")
model2 = cv.fit(df_docs)

In [52]:
# Transform df using model
result = model.transform(df_docs.withColumnRenamed('in_', 'words'))\
        .withColumnRenamed('words', 'in')\
        .withColumnRenamed('vec', 'invec')
result.drop('doc').show(3, True)

# Add a column based on the out column called outvec
result = model2.transform(result.withColumnRenamed('out', 'words'))\
        .withColumnRenamed('words', 'out')\
        .withColumnRenamed('vec', 'outvec')
result.select('invec', 'outvec').show(3,True)

+--------------------+---------+--------------------+
|            words_in|words_out|               invec|
+--------------------+---------+--------------------+
|[the, project, gu...|  [doyle]|(127,[0,4,6,7,8,1...|
|[15, in_, our, se...|  [doyle]|(127,[2,7,14,24,2...|
|[copyright, laws,...|  [world]|(127,[0,15,28,33,...|
+--------------------+---------+--------------------+
only showing top 3 rows

+--------------------+---------------+
|               invec|         outvec|
+--------------------+---------------+
|(127,[0,4,6,7,8,1...| (12,[2],[1.0])|
|(127,[2,7,14,24,2...| (12,[2],[1.0])|
|(127,[0,15,28,33,...|(12,[11],[1.0])|
+--------------------+---------------+
only showing top 3 rows



#### label data

In [53]:
schema = T.StructType([T.StructField('endword', T.StringType()),\
                       T.StructField('doc', T.ArrayType(T.StringType())),\
                       T.StructField('features', VectorUDT()),\
                       T.StructField('outvec', VectorUDT())])

In [54]:
df_class = spark\
.createDataFrame(\
[('it', ['please', 'do', 'not', 'remove', 'it'], SparseVector(12847, {15: 1.0, 47: 1.0, 502: 1.0, 1515: 1.0}), SparseVector(12847, {7: 1.0})),
 ('holmes', ['start', 'of', 'the', 'project', 'gutenberg', 'ebook', 'the', 'adventures', 'of', 'sherlock', 'holmes'], SparseVector(12847, {0: 1.0, 3: 1.0, 183: 1.0, 191: 1.0, 569: 1.0, 1584: 1.0, 1921: 1.0, 3302: 1.0}), SparseVector(12847, {145: 1.0})),
 ('i', ['the', 'adventures', 'of', 'sherlock', 'holmes', 'by', 'sir', 'arthur', 'conan', 'doyle', 'contents', 'i'], SparseVector(12847, {0: 1.0, 3: 1.0, 35: 1.0, 145: 1.0, 569: 1.0, 776: 1.0, 3270: 1.0, 3302: 1.0, 3647: 1.0, 8569: 1.0, 12351: 1.0}), SparseVector(12847, {11: 1.0})),
 ('i', ['the', 'adventure', 'of', 'the', 'copper', 'beeches', 'adventure', 'i'], SparseVector(12847, {0: 1.0, 3: 1.0, 3766: 1.0, 3830: 1.0, 6900: 1.0}), SparseVector(12847, {11: 1.0})),
 ('i', ['a', 'scandal', 'in', 'bohemia', 'i'], SparseVector(12847, {4: 1.0, 5: 1.0, 3669: 1.0, 5237: 1.0}), SparseVector(12847, {11: 1.0})),
 ('well', ['as', 'i', 'passed', 'the', 'well'], SparseVector(12847, {0: 1.0, 11: 1.0, 24: 1.0, 277: 1.0}), SparseVector(12847, {61: 1.0})),
 ('him', ['he', 'was', 'pacing', 'the', 'room', 'swiftly', 'eagerly', 'with', 'his', 'head', 'sunk', 'upon', 'his', 'chest', 'and', 'his', 'hands', 'clasped', 'behind', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 6: 1.0, 10: 1.0, 12: 1.0, 14: 1.0, 62: 1.0, 108: 1.0, 131: 1.0, 227: 1.0, 303: 1.0, 1737: 1.0, 1883: 1.0, 1941: 1.0, 1995: 1.0, 4485: 1.0, 5201: 1.0}), SparseVector(12847, {9: 1.0})),
 ('again', ['he', 'was', 'at', 'work', 'again'], SparseVector(12847, {6: 1.0, 10: 1.0, 18: 1.0, 369: 1.0}), SparseVector(12847, {91: 1.0})),
 ('me', ['but', 'he', 'was', 'glad', 'i', 'think', 'to', 'see', 'me'], SparseVector(12847, {2: 1.0, 6: 1.0, 10: 1.0, 11: 1.0, 19: 1.0, 87: 1.0, 159: 1.0, 420: 1.0}), SparseVector(12847, {34: 1.0})),
 ('you', ['i', 'think', 'watson', 'that', 'you', 'have', 'put', 'on', 'seven', 'and', 'a', 'half', 'pounds', 'since', 'i', 'saw', 'you'], SparseVector(12847, {1: 1.0, 4: 1.0, 8: 1.0, 11: 1.0, 13: 1.0, 20: 1.0, 30: 1.0, 159: 1.0, 192: 1.0, 231: 1.0, 353: 1.0, 506: 1.0, 767: 1.0, 1081: 1.0, 4490: 1.0}), SparseVector(12847, {13: 1.0})),
 ('know', ['then', 'how', 'do', 'you', 'know'], SparseVector(12847, {13: 1.0, 47: 1.0, 60: 1.0, 76: 1.0}), SparseVector(12847, {65: 1.0})),
 ('it', ['i', 'see', 'it', 'i', 'deduce', 'it'], SparseVector(12847, {7: 1.0, 11: 1.0, 87: 1.0, 2320: 1.0}), SparseVector(12847, {7: 1.0})),
 ('it', ['it', 'is', 'true', 'that', 'i', 'had', 'a', 'country', 'walk', 'on', 'thursday', 'and', 'came', 'home', 'in', 'a', 'dreadful', 'mess', 'but', 'as', 'i', 'have', 'changed', 'my', 'clothes', 'i', 'canot', 'imagine', 'how', 'you', 'deduce', 'it'], SparseVector(12847, {1: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 8: 1.0, 11: 1.0, 13: 1.0, 16: 1.0, 17: 1.0, 19: 1.0, 20: 1.0, 24: 1.0, 30: 1.0, 64: 1.0, 76: 1.0, 105: 1.0, 164: 1.0, 333: 1.0, 402: 1.0, 543: 1.0, 650: 1.0, 829: 1.0, 983: 1.0, 1155: 1.0, 1205: 1.0, 2320: 1.0, 5670: 1.0, 6863: 1.0}), SparseVector(12847, {7: 1.0})),
 ('out', ['as', 'to', 'mary', 'jane', 'she', 'is', 'incorrigible', 'and', 'my', 'wife', 'has', 'given', 'her', 'notice', 'but', 'there', 'again', 'i', 'fail', 'to', 'see', 'how', 'you', 'work', 'it', 'out'], SparseVector(12847, {1: 1.0, 2: 1.0, 7: 1.0, 11: 1.0, 13: 1.0, 16: 1.0, 19: 1.0, 21: 1.0, 24: 1.0, 26: 1.0, 38: 1.0, 64: 1.0, 76: 1.0, 83: 1.0, 87: 1.0, 91: 1.0, 143: 1.0, 266: 1.0, 355: 1.0, 369: 1.0, 719: 1.0, 2591: 1.0, 7625: 1.0, 8032: 1.0}), SparseVector(12847, {48: 1.0})),
 ('he', ['it', 'is', 'simplicity', 'itself', 'said', 'he'], SparseVector(12847, {7: 1.0, 16: 1.0, 23: 1.0, 436: 1.0, 3642: 1.0}), SparseVector(12847, {6: 1.0})),
 ('it', ['obviously', 'they', 'have', 'been', 'caused', 'by', 'someone', 'who', 'has', 'very', 'carelessly', 'scraped', 'round', 'the', 'edges', 'of', 'the', 'sole', 'in', 'order', 'to', 'remove', 'crusted', 'mud', 'from', 'it'], SparseVector(12847, {0: 1.0, 2: 1.0, 3: 1.0, 5: 1.0, 28: 1.0, 30: 1.0, 35: 1.0, 40: 1.0, 41: 1.0, 55: 1.0, 83: 1.0, 84: 1.0, 208: 1.0, 324: 1.0, 520: 1.0, 999: 1.0, 1515: 1.0, 1671: 1.0, 1777: 1.0, 2349: 1.0, 3048: 1.0, 3668: 1.0, 8444: 1.0, 12689: 1.0}), SparseVector(12847, {7: 1.0})),
 ('room', ['for', 'example', 'you', 'have', 'frequently', 'seen', 'the', 'steps', 'which', 'lead', 'up', 'from', 'the', 'hall', 'to', 'this', 'room'], SparseVector(12847, {0: 1.0, 2: 1.0, 13: 1.0, 22: 1.0, 28: 1.0, 29: 1.0, 30: 1.0, 39: 1.0, 43: 1.0, 249: 1.0, 561: 1.0, 699: 1.0, 740: 1.0, 1116: 1.0, 1565: 1.0}), SparseVector(12847, {62: 1.0})),
 ('there', ['then', 'how', 'many', 'are', 'there'], SparseVector(12847, {52: 1.0, 60: 1.0, 76: 1.0, 230: 1.0}), SparseVector(12847, {38: 1.0})),
 ('know', ['i', 'donot', 'know'], SparseVector(12847, {11: 1.0, 133: 1.0}), SparseVector(12847, {65: 1.0})),
 ('so', ['quite', 'so'], SparseVector(12847, {203: 1.0}), SparseVector(12847, {32: 1.0})),
 ('this', ['by', 'the', 'way', 'since', 'you', 'are', 'interested', 'in', 'these', 'little', 'problems', 'and', 'since', 'you', 'are', 'good', 'enough', 'to', 'chronicle', 'one', 'or', 'two', 'of', 'my', 'trifling', 'experiences', 'you', 'may', 'be', 'interested', 'in', 'this'], SparseVector(12847, {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 5: 1.0, 13: 1.0, 27: 1.0, 35: 1.0, 36: 1.0, 44: 1.0, 52: 1.0, 64: 1.0, 86: 1.0, 99: 1.0, 106: 1.0, 130: 1.0, 137: 1.0, 156: 1.0, 353: 1.0, 687: 1.0, 1191: 1.0, 1837: 1.0, 3556: 1.0, 4435: 1.0, 10616: 1.0}), SparseVector(12847, {29: 1.0})),
 ('he', ['it', 'came', 'by', 'the', 'last', 'post', 'said', 'he'], SparseVector(12847, {0: 1.0, 7: 1.0, 23: 1.0, 35: 1.0, 105: 1.0, 225: 1.0, 1035: 1.0}), SparseVector(12847, {6: 1.0})),
 ('to', ['there', 'will', 'call', 'upon', 'you', 'to'], SparseVector(12847, {13: 1.0, 38: 1.0, 68: 1.0, 131: 1.0, 514: 1.0}), SparseVector(12847, {2: 1.0})),
 ('it', ['what', 'do', 'you', 'deduce', 'from', 'it'], SparseVector(12847, {13: 1.0, 28: 1.0, 31: 1.0, 47: 1.0, 2320: 1.0}), SparseVector(12847, {7: 1.0})),
 ('holmes', ['that', 'is', 'the', 'very', 'word', 'said', 'holmes'], SparseVector(12847, {0: 1.0, 8: 1.0, 16: 1.0, 23: 1.0, 84: 1.0, 340: 1.0}), SparseVector(12847, {145: 1.0})),
 ('all', ['it', 'is', 'not', 'an', 'english', 'paper', 'at', 'all'], SparseVector(12847, {7: 1.0, 15: 1.0, 16: 1.0, 18: 1.0, 51: 1.0, 796: 1.0, 918: 1.0}), SparseVector(12847, {25: 1.0})),
 ('that', ['what', 'do', 'you', 'make', 'of', 'that'], SparseVector(12847, {3: 1.0, 13: 1.0, 31: 1.0, 47: 1.0, 218: 1.0}), SparseVector(12847, {8: 1.0})),
 ('holmes', ['asked', 'holmes'], SparseVector(12847, {80: 1.0}), SparseVector(12847, {145: 1.0})),
 ('all', ['not', 'at', 'all'], SparseVector(12847, {15: 1.0, 18: 1.0}), SparseVector(12847, {25: 1.0})),
 ('country', ['speaking', 'country'], SparseVector(12847, {367: 1.0}), SparseVector(12847, {164: 1.0})),
 ('that', ['ha', 'ha', 'my', 'boy', 'what', 'do', 'you', 'make', 'of', 'that'], SparseVector(12847, {3: 1.0, 13: 1.0, 31: 1.0, 47: 1.0, 64: 1.0, 218: 1.0, 578: 1.0, 6157: 1.0}), SparseVector(12847, {8: 1.0})),
 ('said', ['the', 'paper', 'was', 'made', 'in', 'bohemia', 'i', 'said'], SparseVector(12847, {0: 1.0, 5: 1.0, 10: 1.0, 11: 1.0, 128: 1.0, 796: 1.0, 5237: 1.0}), SparseVector(12847, {23: 1.0})),
 ('that', ['a', 'frenchman', 'or', 'russian', 'could', 'not', 'have', 'written', 'that'], SparseVector(12847, {4: 1.0, 15: 1.0, 30: 1.0, 44: 1.0, 73: 1.0, 239: 1.0, 576: 1.0, 916: 1.0}), SparseVector(12847, {8: 1.0})),
 ('face', ['it', 'only', 'remains', 'therefore', 'to', 'discover', 'what', 'is', 'wanted', 'by', 'this', 'german', 'who', 'writes', 'upon', 'bohemian', 'paper', 'and', 'prefers', 'wearing', 'a', 'mask', 'to', 'showing', 'his', 'face'], SparseVector(12847, {1: 1.0, 2: 1.0, 4: 1.0, 7: 1.0, 14: 1.0, 16: 1.0, 29: 1.0, 31: 1.0, 35: 1.0, 41: 1.0, 71: 1.0, 131: 1.0, 313: 1.0, 796: 1.0, 956: 1.0, 997: 1.0, 1074: 1.0, 1576: 1.0, 1719: 1.0, 2806: 1.0, 4001: 1.0, 5932: 1.0, 12256: 1.0, 12258: 1.0}), SparseVector(12847, {74: 1.0})),
 ('he', ['a', 'pair', 'by', 'the', 'sound', 'said', 'he'], SparseVector(12847, {0: 1.0, 4: 1.0, 23: 1.0, 35: 1.0, 449: 1.0, 2499: 1.0}), SparseVector(12847, {6: 1.0})),
 ('holmes', ['i', 'think', 'that', 'i', 'had', 'better', 'go', 'holmes'], SparseVector(12847, {8: 1.0, 11: 1.0, 17: 1.0, 104: 1.0, 159: 1.0, 323: 1.0}), SparseVector(12847, {145: 1.0})),
 ('it', ['it', 'would', 'be', 'a', 'pity', 'to', 'miss', 'it'], SparseVector(12847, {2: 1.0, 4: 1.0, 7: 1.0, 27: 1.0, 59: 1.0, 822: 1.0, 1287: 1.0}), SparseVector(12847, {7: 1.0})),
 ('him', ['never', 'mind', 'him'], SparseVector(12847, {157: 1.0, 325: 1.0}), SparseVector(12847, {9: 1.0})),
 ('he', ['i', 'may', 'want', 'your', 'help', 'and', 'so', 'may', 'he'], SparseVector(12847, {1: 1.0, 11: 1.0, 32: 1.0, 106: 1.0, 124: 1.0, 246: 1.0, 291: 1.0}), SparseVector(12847, {6: 1.0})),
 ('door', ['a', 'slow', 'and', 'heavy', 'step', 'which', 'had', 'been', 'heard', 'upon', 'the', 'stairs', 'and', 'in', 'the', 'passage', 'paused', 'immediately', 'outside', 'the', 'door'], SparseVector(12847, {0: 1.0, 1: 1.0, 4: 1.0, 5: 1.0, 17: 1.0, 39: 1.0, 55: 1.0, 131: 1.0, 193: 1.0, 549: 1.0, 664: 1.0, 973: 1.0, 1001: 1.0, 1098: 1.0, 1854: 1.0, 2659: 1.0, 4817: 1.0}), SparseVector(12847, {118: 1.0})),
 ('in', ['come', 'in'], SparseVector(12847, {93: 1.0}), SparseVector(12847, {5: 1.0})),
 ('holmes', ['said', 'holmes'], SparseVector(12847, {23: 1.0}), SparseVector(12847, {145: 1.0})),
 ('holmes', ['pray', 'take', 'a', 'seat', 'said', 'holmes'], SparseVector(12847, {4: 1.0, 23: 1.0, 174: 1.0, 712: 1.0, 972: 1.0}), SparseVector(12847, {145: 1.0})),
 ('he', ['it', 'is', 'both', 'or', 'none', 'said', 'he'], SparseVector(12847, {7: 1.0, 16: 1.0, 23: 1.0, 44: 1.0, 268: 1.0, 749: 1.0}), SparseVector(12847, {6: 1.0})),
 ('me', ['you', 'may', 'say', 'before', 'this', 'gentleman', 'anything', 'which', 'you', 'may', 'say', 'to', 'me'], SparseVector(12847, {2: 1.0, 13: 1.0, 29: 1.0, 39: 1.0, 67: 1.0, 81: 1.0, 106: 1.0, 237: 1.0, 1149: 1.0}), SparseVector(12847, {34: 1.0})),
 ('holmes', ['i', 'promise', 'said', 'holmes'], SparseVector(12847, {11: 1.0, 23: 1.0, 1153: 1.0}), SparseVector(12847, {145: 1.0})),
 ('i', ['and', 'i'], SparseVector(12847, {1: 1.0}), SparseVector(12847, {11: 1.0})),
 ('good', ['then', 'i', 'rather', 'imprudently', 'wished', 'you', 'good'], SparseVector(12847, {11: 1.0, 13: 1.0, 60: 1.0, 302: 1.0, 528: 1.0, 9310: 1.0}), SparseVector(12847, {99: 1.0})),
 ('to', ['so', 'you', 'will', 'find', 'the', 'nest', 'empty', 'when', 'you', 'call', 'to'], SparseVector(12847, {0: 1.0, 13: 1.0, 32: 1.0, 45: 1.0, 68: 1.0, 301: 1.0, 514: 1.0, 2058: 1.0, 11076: 1.0}), SparseVector(12847, {2: 1.0})),
 ('he', ['i', 'love', 'and', 'am', 'loved', 'by', 'a', 'better', 'man', 'than', 'he'], SparseVector(12847, {1: 1.0, 4: 1.0, 11: 1.0, 35: 1.0, 53: 1.0, 113: 1.0, 120: 1.0, 166: 1.0, 323: 1.0, 423: 1.0}), SparseVector(12847, {6: 1.0})),
 ('so', ['i', 'am', 'glad', 'to', 'hear', 'your', 'majesty', 'say', 'so'], SparseVector(12847, {2: 1.0, 11: 1.0, 81: 1.0, 120: 1.0, 124: 1.0, 396: 1.0, 420: 1.0, 1124: 1.0}), SparseVector(12847, {32: 1.0})),
 ('you', ['i', 'am', 'immensely', 'indebted', 'to', 'you'], SparseVector(12847, {2: 1.0, 11: 1.0, 120: 1.0, 4039: 1.0, 6011: 1.0}), SparseVector(12847, {13: 1.0})),
 ('you', ['pray', 'tell', 'me', 'in', 'what', 'way', 'i', 'can', 'reward', 'you'], SparseVector(12847, {5: 1.0, 11: 1.0, 31: 1.0, 34: 1.0, 86: 1.0, 129: 1.0, 160: 1.0, 712: 1.0, 2122: 1.0}), SparseVector(12847, {13: 1.0})),
 ('hand', ['he', 'slipped', 'an', 'emerald', 'snake', 'ring', 'from', 'his', 'finger', 'and', 'held', 'it', 'out', 'upon', 'the', 'palm', 'of', 'his', 'hand'], SparseVector(12847, {0: 1.0, 1: 1.0, 3: 1.0, 6: 1.0, 7: 1.0, 14: 1.0, 28: 1.0, 48: 1.0, 51: 1.0, 131: 1.0, 296: 1.0, 1309: 1.0, 1665: 1.0, 1762: 1.0, 2336: 1.0, 5166: 1.0, 7782: 1.0}), SparseVector(12847, {82: 1.0})),
 ('holmes', ['your', 'majesty', 'has', 'something', 'which', 'i', 'should', 'value', 'even', 'more', 'highly', 'said', 'holmes'], SparseVector(12847, {11: 1.0, 23: 1.0, 39: 1.0, 83: 1.0, 92: 1.0, 109: 1.0, 124: 1.0, 142: 1.0, 161: 1.0, 1079: 1.0, 1124: 1.0, 1845: 1.0}), SparseVector(12847, {145: 1.0})),
 ('it', ['you', 'have', 'but', 'to', 'name', 'it'], SparseVector(12847, {2: 1.0, 13: 1.0, 19: 1.0, 30: 1.0, 484: 1.0}), SparseVector(12847, {7: 1.0})),
 ('it', ['certainly', 'if', 'you', 'wish', 'it'], SparseVector(12847, {13: 1.0, 49: 1.0, 257: 1.0, 633: 1.0}), SparseVector(12847, {7: 1.0})),
 ('me', ['with', 'an', 'apology', 'for', 'my', 'intrusion', 'i', 'was', 'about', 'to', 'withdraw', 'when', 'holmes', 'pulled', 'me', 'abruptly', 'into', 'the', 'room', 'and', 'closed', 'the', 'door', 'behind', 'me'], SparseVector(12847, {0: 1.0, 1: 1.0, 2: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 22: 1.0, 34: 1.0, 45: 1.0, 51: 1.0, 62: 1.0, 64: 1.0, 69: 1.0, 70: 1.0, 118: 1.0, 145: 1.0, 227: 1.0, 415: 1.0, 1216: 1.0, 3875: 1.0, 4310: 1.0, 4549: 1.0, 10464: 1.0}), SparseVector(12847, {34: 1.0})),
 ('so', ['very', 'much', 'so'], SparseVector(12847, {84: 1.0, 171: 1.0}), SparseVector(12847, {32: 1.0})),
 ('room', ['then', 'i', 'can', 'wait', 'in', 'the', 'next', 'room'], SparseVector(12847, {0: 1.0, 5: 1.0, 11: 1.0, 60: 1.0, 129: 1.0, 293: 1.0, 660: 1.0}), SparseVector(12847, {62: 1.0})),
 ('all', ['not', 'at', 'all'], SparseVector(12847, {15: 1.0, 18: 1.0}), SparseVector(12847, {25: 1.0})),
 ('eyes', ['encircled', 'eyes'], SparseVector(12847, {10811: 1.0}), SparseVector(12847, {72: 1.0})),
 ('i', ['the', 'adventure', 'of', 'the', 'copper', 'beeches', 'adventure', 'i'], SparseVector(12847, {0: 1.0, 3: 1.0, 3766: 1.0, 3830: 1.0, 6900: 1.0}), SparseVector(12847, {11: 1.0})),
 ('he', ['it', 'is', 'simplicity', 'itself', 'said', 'he'], SparseVector(12847, {7: 1.0, 16: 1.0, 23: 1.0, 436: 1.0, 3642: 1.0}), SparseVector(12847, {6: 1.0})),
 ('it', ['why', 'should', 'i', 'attempt', 'to', 'conceal', 'it'], SparseVector(12847, {2: 1.0, 11: 1.0, 109: 1.0, 140: 1.0, 1469: 1.0, 1738: 1.0}), SparseVector(12847, {7: 1.0})),
 ('it', ['and', 'she', 'will', 'do', 'it'], SparseVector(12847, {1: 1.0, 26: 1.0, 47: 1.0, 68: 1.0}), SparseVector(12847, {7: 1.0})),
 ('said', ['there', 'are', 'three', 'hundred', 'pounds', 'in', 'gold', 'and', 'seven', 'hundred', 'in', 'notes', 'he', 'said'], SparseVector(12847, {1: 1.0, 5: 1.0, 6: 1.0, 38: 1.0, 52: 1.0, 314: 1.0, 560: 1.0, 767: 1.0, 1369: 1.0, 2476: 1.0, 4490: 1.0}), SparseVector(12847, {23: 1.0})),
 ('life', ['i', 'know', 'my', 'dear', 'watson', 'that', 'you', 'share', 'my', 'love', 'of', 'all', 'that', 'is', 'bizarre', 'and', 'outside', 'the', 'conventions', 'and', 'humdrum', 'routine', 'of', 'everyday', 'life'], SparseVector(12847, {0: 1.0, 1: 1.0, 3: 1.0, 8: 1.0, 11: 1.0, 13: 1.0, 16: 1.0, 25: 1.0, 64: 1.0, 65: 1.0, 166: 1.0, 263: 1.0, 664: 1.0, 1081: 1.0, 1245: 1.0, 3981: 1.0, 4124: 1.0, 4266: 1.0, 8342: 1.0, 11111: 1.0}), SparseVector(12847, {79: 1.0})),
 ('right', ['you', 'did', 'doctor', 'but', 'none', 'the', 'less', 'you', 'must', 'come', 'round', 'to', 'my', 'view', 'for', 'otherwise', 'i', 'shall', 'keep', 'on', 'piling', 'fact', 'upon', 'fact', 'on', 'you', 'until', 'your', 'reason', 'breaks', 'down', 'under', 'them', 'and', 'acknowledges', 'me', 'to', 'be', 'right'], SparseVector(12847, {0: 1.0, 1: 1.0, 2: 1.0, 11: 1.0, 13: 1.0, 19: 1.0, 20: 1.0, 22: 1.0, 27: 1.0, 33: 1.0, 34: 1.0, 50: 1.0, 64: 1.0, 85: 1.0, 93: 1.0, 124: 1.0, 131: 1.0, 152: 1.0, 176: 1.0, 184: 1.0, 208: 1.0, 393: 1.0, 451: 1.0, 463: 1.0, 467: 1.0, 579: 1.0, 651: 1.0, 749: 1.0, 757: 1.0, 2033: 1.0, 4396: 1.0, 7049: 1.0, 12739: 1.0}), SparseVector(12847, {117: 1.0}))],\
 schema=schema)


In [55]:
# Select the rows where endword is 'him' and label 1
df_pos = df_class.where("endword = 'him'")\
           .withColumn('label', F.lit(1))

# Select the rows where endword is not 'him' and label 0
df_neg = df_class.where("endword <> 'him'")\
           .withColumn('label', F.lit(0))

# Union pos and neg in equal number
df_examples = df_pos.union(df_neg.limit(df_pos.count()))
print("Number of examples: ", df_examples.count())
df_examples.where("endword <> 'him'").show(5)

Number of examples:  4
+-------+--------------------+--------------------+-------------------+-----+
|endword|                 doc|            features|             outvec|label|
+-------+--------------------+--------------------+-------------------+-----+
|     it|[please, do, not,...|(12847,[15,47,502...|  (12847,[7],[1.0])|    0|
| holmes|[start, of, the, ...|(12847,[0,3,183,1...|(12847,[145],[1.0])|    0|
+-------+--------------------+--------------------+-------------------+-----+



#### split

In [56]:
schema = T.StructType([T.StructField('endword', T.StringType()),\
                       T.StructField('doc', T.ArrayType(T.StringType())),\
                       T.StructField('features', VectorUDT()),\
                       T.StructField('outvec', VectorUDT()),\
                       T.StructField('label', T.IntegerType())])

In [57]:
df_model = spark\
.createDataFrame(\
[('him', ['he', 'was', 'pacing', 'the', 'room', 'swiftly', 'eagerly', 'with', 'his', 'head', 'sunk', 'upon', 'his', 'chest', 'and', 'his', 'hands', 'clasped', 'behind', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 6: 1.0, 10: 1.0, 12: 1.0, 14: 1.0, 62: 1.0, 108: 1.0, 131: 1.0, 227: 1.0, 303: 1.0, 1737: 1.0, 1883: 1.0, 1941: 1.0, 1995: 1.0, 4485: 1.0, 5201: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['never', 'mind', 'him'], SparseVector(12847, {157: 1.0, 325: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['book', 'and', 'handed', 'it', 'to', 'him'], SparseVector(12847, {1: 1.0, 2: 1.0, 7: 1.0, 799: 1.0, 1285: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['has', 'only', 'one', 'male', 'visitor', 'but', 'a', 'good', 'deal', 'of', 'him'], SparseVector(12847, {3: 1.0, 4: 1.0, 19: 1.0, 36: 1.0, 71: 1.0, 83: 1.0, 99: 1.0, 1033: 1.0, 1657: 1.0, 1843: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['mews', 'and', 'knew', 'all', 'about', 'him'], SparseVector(12847, {1: 1.0, 25: 1.0, 69: 1.0, 190: 1.0, 5920: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['i', 'shall', 'drive', 'out', 'in', 'the', 'park', 'at', 'five', 'as', 'usual', 'she', 'said', 'as', 'she', 'left', 'him'], SparseVector(12847, {0: 1.0, 5: 1.0, 11: 1.0, 18: 1.0, 23: 1.0, 24: 1.0, 26: 1.0, 48: 1.0, 136: 1.0, 176: 1.0, 508: 1.0, 689: 1.0, 702: 1.0, 9693: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['no', 'no', 'there', 's', 'life', 'in', 'him'], SparseVector(12847, {5: 1.0, 38: 1.0, 54: 1.0, 79: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['they', 'would', 'have', 'had', 'the', 'lady', 's', 'purse', 'and', 'watch', 'if', 'it', 'hadnot', 'been', 'for', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 7: 1.0, 17: 1.0, 22: 1.0, 30: 1.0, 40: 1.0, 49: 1.0, 55: 1.0, 59: 1.0, 397: 1.0, 1972: 1.0, 1978: 1.0, 3013: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['but', 'she', 'could', 'not', 'love', 'him'], SparseVector(12847, {15: 1.0, 19: 1.0, 26: 1.0, 73: 1.0, 166: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['hat', 'and', 'a', 'faded', 'brown', 'overcoat', 'with', 'a', 'wrinkled', 'velvet', 'collar', 'lay', 'upon', 'a', 'chair', 'beside', 'him'], SparseVector(12847, {1: 1.0, 4: 1.0, 12: 1.0, 131: 1.0, 287: 1.0, 288: 1.0, 752: 1.0, 1006: 1.0, 2900: 1.0, 3951: 1.0, 4273: 1.0, 4781: 1.0, 5134: 1.0, 9128: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['and', 'i', 'know', 'very', 'well', 'that', 'he', 'could', 'better', 'himself', 'and', 'earn', 'twice', 'what', 'i', 'am', 'able', 'to', 'give', 'him'], SparseVector(12847, {1: 1.0, 2: 1.0, 6: 1.0, 8: 1.0, 11: 1.0, 31: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 73: 1.0, 84: 1.0, 120: 1.0, 242: 1.0, 323: 1.0, 448: 1.0, 1072: 1.0, 4891: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['there', 's', 'no', 'vice', 'in', 'him'], SparseVector(12847, {5: 1.0, 38: 1.0, 54: 1.0, 10753: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['he', 'answered', 'that', 'the', 'name', 'was', 'new', 'to', 'him'], SparseVector(12847, {0: 1.0, 2: 1.0, 6: 1.0, 8: 1.0, 10: 1.0, 188: 1.0, 479: 1.0, 484: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['where', 'could', 'i', 'find', 'him'], SparseVector(12847, {11: 1.0, 73: 1.0, 151: 1.0, 301: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['why', 'did', 'you', 'pick', 'him'], SparseVector(12847, {13: 1.0, 50: 1.0, 140: 1.0, 2520: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['i', 'have', 'only', 'just', 'left', 'him'], SparseVector(12847, {11: 1.0, 30: 1.0, 71: 1.0, 136: 1.0, 144: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['i', 'am', 'sure', 'that', 'you', 'inquired', 'your', 'way', 'merely', 'in', 'order', 'that', 'you', 'might', 'see', 'him'], SparseVector(12847, {5: 1.0, 8: 1.0, 11: 1.0, 13: 1.0, 86: 1.0, 87: 1.0, 120: 1.0, 124: 1.0, 195: 1.0, 324: 1.0, 428: 1.0, 468: 1.0, 1902: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['not', 'him'], SparseVector(12847, {15: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['in', 'his', 'singular', 'character', 'the', 'dual', 'nature', 'alternately', 'asserted', 'itself', 'and', 'his', 'extreme', 'exactness', 'and', 'astuteness', 'represented', 'as', 'i', 'have', 'often', 'thought', 'the', 'reaction', 'against', 'the', 'poetic', 'and', 'contemplative', 'mood', 'which', 'occasionally', 'predominated', 'in', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 5: 1.0, 11: 1.0, 14: 1.0, 24: 1.0, 30: 1.0, 39: 1.0, 102: 1.0, 213: 1.0, 354: 1.0, 436: 1.0, 886: 1.0, 1140: 1.0, 1193: 1.0, 1405: 1.0, 2166: 1.0, 2173: 1.0, 2314: 1.0, 2825: 1.0, 3493: 1.0, 5568: 1.0, 7470: 1.0, 9317: 1.0, 10383: 1.0, 10685: 1.0, 11812: 1.0, 12002: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['he', 'has', 'his', 'own', 'little', 'methods', 'which', 'are', 'if', 'he', 'wonot', 'mind', 'my', 'saying', 'so', 'just', 'a', 'little', 'too', 'theoretical', 'and', 'fantastic', 'but', 'he', 'has', 'the', 'makings', 'of', 'a', 'detective', 'in', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 14: 1.0, 19: 1.0, 32: 1.0, 39: 1.0, 49: 1.0, 52: 1.0, 64: 1.0, 83: 1.0, 137: 1.0, 144: 1.0, 167: 1.0, 248: 1.0, 325: 1.0, 330: 1.0, 398: 1.0, 2831: 1.0, 3741: 1.0, 10855: 1.0, 11837: 1.0, 12806: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['you', 'are', 'not', 'very', 'vulnerable', 'from', 'above', 'holmes', 'remarked', 'as', 'he', 'held', 'up', 'the', 'lantern', 'and', 'gazed', 'about', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 6: 1.0, 13: 1.0, 15: 1.0, 24: 1.0, 28: 1.0, 43: 1.0, 52: 1.0, 69: 1.0, 84: 1.0, 145: 1.0, 296: 1.0, 381: 1.0, 581: 1.0, 743: 1.0, 7397: 1.0, 10657: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['sherlock', 'holmes', 'welcomed', 'her', 'with', 'the', 'easy', 'courtesy', 'for', 'which', 'he', 'was', 'remarkable', 'and', 'having', 'closed', 'the', 'door', 'and', 'bowed', 'her', 'into', 'an', 'armchair', 'he', 'looked', 'her', 'over', 'in', 'the', 'minute', 'and', 'yet', 'abstracted', 'fashion', 'which', 'was', 'peculiar', 'to', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 2: 1.0, 5: 1.0, 6: 1.0, 10: 1.0, 12: 1.0, 21: 1.0, 22: 1.0, 39: 1.0, 51: 1.0, 70: 1.0, 116: 1.0, 118: 1.0, 121: 1.0, 145: 1.0, 181: 1.0, 251: 1.0, 415: 1.0, 569: 1.0, 913: 1.0, 958: 1.0, 1041: 1.0, 1236: 1.0, 1581: 1.0, 1881: 1.0, 3396: 1.0, 4878: 1.0, 6148: 1.0, 6623: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['i', 'met', 'him', 'that', 'night', 'and', 'he', 'called', 'next', 'day', 'to', 'ask', 'if', 'we', 'had', 'got', 'home', 'all', 'safe', 'and', 'after', 'that', 'we', 'met', 'him'], SparseVector(12847, {1: 1.0, 2: 1.0, 6: 1.0, 8: 1.0, 9: 1.0, 11: 1.0, 17: 1.0, 25: 1.0, 49: 1.0, 75: 1.0, 100: 1.0, 111: 1.0, 267: 1.0, 292: 1.0, 293: 1.0, 298: 1.0, 315: 1.0, 339: 1.0, 402: 1.0, 2390: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['he', 'was', 'in', 'dreadful', 'earnest', 'and', 'made', 'me', 'swear', 'with', 'my', 'hands', 'on', 'the', 'testament', 'that', 'whatever', 'happened', 'i', 'would', 'always', 'be', 'true', 'to', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 2: 1.0, 5: 1.0, 6: 1.0, 8: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 20: 1.0, 27: 1.0, 34: 1.0, 59: 1.0, 64: 1.0, 128: 1.0, 194: 1.0, 303: 1.0, 404: 1.0, 543: 1.0, 930: 1.0, 1155: 1.0, 4265: 1.0, 8801: 1.0, 11234: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['but', 'they', 'both', 'said', 'never', 'to', 'mind', 'about', 'father', 'but', 'just', 'to', 'tell', 'him', 'afterwards', 'and', 'mother', 'said', 'she', 'would', 'make', 'it', 'all', 'right', 'with', 'him'], SparseVector(12847, {1: 1.0, 2: 1.0, 7: 1.0, 9: 1.0, 12: 1.0, 19: 1.0, 23: 1.0, 25: 1.0, 26: 1.0, 40: 1.0, 59: 1.0, 69: 1.0, 117: 1.0, 144: 1.0, 157: 1.0, 160: 1.0, 212: 1.0, 218: 1.0, 268: 1.0, 280: 1.0, 325: 1.0, 1385: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['that', 'was', 'last', 'friday', 'mr', 'holmes', 'and', 'i', 'have', 'never', 'seen', 'or', 'heard', 'anything', 'since', 'then', 'to', 'throw', 'any', 'light', 'upon', 'what', 'became', 'of', 'him'], SparseVector(12847, {1: 1.0, 2: 1.0, 3: 1.0, 8: 1.0, 10: 1.0, 11: 1.0, 30: 1.0, 31: 1.0, 44: 1.0, 60: 1.0, 125: 1.0, 131: 1.0, 145: 1.0, 157: 1.0, 193: 1.0, 225: 1.0, 237: 1.0, 249: 1.0, 252: 1.0, 353: 1.0, 414: 1.0, 431: 1.0, 2553: 1.0, 4284: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['your', 'own', 'opinion', 'is', 'then', 'that', 'some', 'unforeseen', 'catastrophe', 'has', 'occurred', 'to', 'him'], SparseVector(12847, {2: 1.0, 8: 1.0, 16: 1.0, 60: 1.0, 83: 1.0, 96: 1.0, 124: 1.0, 167: 1.0, 838: 1.0, 890: 1.0, 8739: 1.0, 10507: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['did', 'you', 'tell', 'him'], SparseVector(12847, {13: 1.0, 50: 1.0, 160: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['then', 'what', 'has', 'happened', 'to', 'him'], SparseVector(12847, {2: 1.0, 31: 1.0, 60: 1.0, 83: 1.0, 404: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['here', 'is', 'the', 'slip', 'and', 'here', 'are', 'four', 'letters', 'from', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 16: 1.0, 28: 1.0, 52: 1.0, 97: 1.0, 442: 1.0, 810: 1.0, 4094: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['tubes', 'with', 'the', 'pungent', 'cleanly', 'smell', 'of', 'hydrochloric', 'acid', 'told', 'me', 'that', 'he', 'had', 'spent', 'his', 'day', 'in', 'the', 'chemical', 'work', 'which', 'was', 'so', 'dear', 'to', 'him'], SparseVector(12847, {0: 1.0, 2: 1.0, 3: 1.0, 5: 1.0, 6: 1.0, 8: 1.0, 10: 1.0, 12: 1.0, 14: 1.0, 17: 1.0, 32: 1.0, 34: 1.0, 39: 1.0, 111: 1.0, 186: 1.0, 263: 1.0, 369: 1.0, 706: 1.0, 2471: 1.0, 4512: 1.0, 5532: 1.0, 5916: 1.0, 6501: 1.0, 9365: 1.0, 12459: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['i', 'let', 'you', 'know', 'then', 'that', 'i', 'have', 'caught', 'him'], SparseVector(12847, {8: 1.0, 11: 1.0, 13: 1.0, 30: 1.0, 60: 1.0, 65: 1.0, 173: 1.0, 949: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['we', 'had', 'the', 'carriage', 'to', 'ourselves', 'save', 'for', 'an', 'immense', 'litter', 'of', 'papers', 'which', 'holmes', 'had', 'brought', 'with', 'him'], SparseVector(12847, {0: 1.0, 2: 1.0, 3: 1.0, 12: 1.0, 17: 1.0, 22: 1.0, 39: 1.0, 51: 1.0, 75: 1.0, 145: 1.0, 250: 1.0, 604: 1.0, 1040: 1.0, 1168: 1.0, 1378: 1.0, 1402: 1.0, 8533: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['to', 'the', 'best', 'of', 'his', 'belief', 'the', 'father', 'was', 'actually', 'in', 'sight', 'at', 'the', 'time', 'and', 'the', 'son', 'was', 'following', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 5: 1.0, 10: 1.0, 14: 1.0, 18: 1.0, 58: 1.0, 212: 1.0, 328: 1.0, 446: 1.0, 517: 1.0, 648: 1.0, 1516: 1.0, 4473: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['however', 'innocent', 'he', 'might', 'be', 'he', 'could', 'not', 'be', 'such', 'an', 'absolute', 'imbecile', 'as', 'not', 'to', 'see', 'that', 'the', 'circumstances', 'were', 'very', 'black', 'against', 'him'], SparseVector(12847, {0: 1.0, 2: 1.0, 6: 1.0, 8: 1.0, 15: 1.0, 24: 1.0, 27: 1.0, 42: 1.0, 51: 1.0, 73: 1.0, 84: 1.0, 87: 1.0, 134: 1.0, 195: 1.0, 213: 1.0, 295: 1.0, 457: 1.0, 1262: 1.0, 1961: 1.0, 2255: 1.0, 7683: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['as', 'to', 'his', 'remark', 'about', 'his', 'deserts', 'it', 'was', 'also', 'not', 'unnatural', 'if', 'you', 'consider', 'that', 'he', 'stood', 'beside', 'the', 'dead', 'body', 'of', 'his', 'father', 'and', 'that', 'there', 'is', 'no', 'doubt', 'that', 'he', 'had', 'that', 'very', 'day', 'so', 'far', 'forgotten', 'his', 'filial', 'duty', 'as', 'to', 'bandy', 'words', 'with', 'him', 'and', 'even', 'according', 'to', 'the', 'little', 'girl', 'whose', 'evidence', 'is', 'so', 'important', 'to', 'raise', 'his', 'hand', 'as', 'if', 'to', 'strike', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 17: 1.0, 24: 1.0, 32: 1.0, 38: 1.0, 49: 1.0, 54: 1.0, 69: 1.0, 82: 1.0, 84: 1.0, 111: 1.0, 137: 1.0, 142: 1.0, 212: 1.0, 219: 1.0, 235: 1.0, 283: 1.0, 287: 1.0, 322: 1.0, 487: 1.0, 498: 1.0, 504: 1.0, 512: 1.0, 637: 1.0, 734: 1.0, 779: 1.0, 792: 1.0, 844: 1.0, 1129: 1.0, 1217: 1.0, 2045: 1.0, 2132: 1.0, 2311: 1.0, 2936: 1.0, 4690: 1.0, 11578: 1.0, 11632: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['such', 'a', 'charge', 'is', 'absurd', 'to', 'anyone', 'who', 'really', 'knows', 'him'], SparseVector(12847, {2: 1.0, 4: 1.0, 16: 1.0, 41: 1.0, 134: 1.0, 276: 1.0, 411: 1.0, 696: 1.0, 1126: 1.0, 4387: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['i', 'must', 'go', 'home', 'now', 'for', 'dad', 'is', 'very', 'ill', 'and', 'he', 'misses', 'me', 'so', 'if', 'i', 'leave', 'him'], SparseVector(12847, {1: 1.0, 6: 1.0, 11: 1.0, 16: 1.0, 22: 1.0, 32: 1.0, 34: 1.0, 49: 1.0, 57: 1.0, 84: 1.0, 104: 1.0, 152: 1.0, 270: 1.0, 402: 1.0, 1495: 1.0, 5458: 1.0, 10422: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['and', 'what', 'did', 'you', 'learn', 'from', 'him'], SparseVector(12847, {1: 1.0, 13: 1.0, 28: 1.0, 31: 1.0, 50: 1.0, 2432: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['this', 'business', 'has', 'had', 'a', 'very', 'bad', 'effect', 'upon', 'him'], SparseVector(12847, {4: 1.0, 17: 1.0, 29: 1.0, 83: 1.0, 84: 1.0, 131: 1.0, 412: 1.0, 587: 1.0, 833: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['in', 'a', 'hundred', 'other', 'ways', 'he', 'has', 'helped', 'him'], SparseVector(12847, {4: 1.0, 5: 1.0, 6: 1.0, 83: 1.0, 126: 1.0, 560: 1.0, 1859: 1.0, 2451: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['everybody', 'about', 'here', 'speaks', 'of', 'his', 'kindness', 'to', 'him'], SparseVector(12847, {2: 1.0, 3: 1.0, 14: 1.0, 69: 1.0, 97: 1.0, 580: 1.0, 3789: 1.0, 4789: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['men', 'who', 'had', 'only', 'known', 'the', 'quiet', 'thinker', 'and', 'logician', 'of', 'baker', 'street', 'would', 'have', 'failed', 'to', 'recognise', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 17: 1.0, 30: 1.0, 41: 1.0, 59: 1.0, 71: 1.0, 95: 1.0, 382: 1.0, 657: 1.0, 922: 1.0, 1480: 1.0, 2243: 1.0, 6586: 1.0, 10824: 1.0, 12664: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['well', 'now', 'in', 'considering', 'this', 'case', 'there', 'are', 'two', 'points', 'about', 'young', 'mccarthy', 's', 'narrative', 'which', 'struck', 'us', 'both', 'instantly', 'although', 'they', 'impressed', 'me', 'in', 'his', 'favour', 'and', 'you', 'against', 'him'], SparseVector(12847, {1: 1.0, 5: 1.0, 13: 1.0, 14: 1.0, 29: 1.0, 34: 1.0, 38: 1.0, 39: 1.0, 40: 1.0, 52: 1.0, 57: 1.0, 61: 1.0, 69: 1.0, 112: 1.0, 156: 1.0, 196: 1.0, 213: 1.0, 268: 1.0, 400: 1.0, 550: 1.0, 1015: 1.0, 1456: 1.0, 1661: 1.0, 2106: 1.0, 2682: 1.0, 2841: 1.0, 4312: 1.0, 5208: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['before', 'seeing', 'him'], SparseVector(12847, {67: 1.0, 363: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['holmes', 'i', 'said', 'you', 'have', 'drawn', 'a', 'net', 'round', 'this', 'man', 'from', 'which', 'he', 'cannot', 'escape', 'and', 'you', 'have', 'saved', 'an', 'innocent', 'human', 'life', 'as', 'truly', 'as', 'if', 'you', 'had', 'cut', 'the', 'cord', 'which', 'was', 'hanging', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 4: 1.0, 6: 1.0, 10: 1.0, 11: 1.0, 13: 1.0, 17: 1.0, 23: 1.0, 24: 1.0, 28: 1.0, 29: 1.0, 30: 1.0, 39: 1.0, 49: 1.0, 51: 1.0, 53: 1.0, 79: 1.0, 145: 1.0, 208: 1.0, 331: 1.0, 618: 1.0, 640: 1.0, 811: 1.0, 1201: 1.0, 1484: 1.0, 1549: 1.0, 1961: 1.0, 2400: 1.0, 3740: 1.0, 8682: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['holmes', 'rose', 'and', 'sat', 'down', 'at', 'the', 'table', 'with', 'his', 'pen', 'in', 'his', 'hand', 'and', 'a', 'bundle', 'of', 'paper', 'before', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 12: 1.0, 14: 1.0, 18: 1.0, 67: 1.0, 82: 1.0, 85: 1.0, 145: 1.0, 223: 1.0, 389: 1.0, 408: 1.0, 796: 1.0, 4489: 1.0, 12014: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['some', 'too', 'have', 'baffled', 'his', 'analytical', 'skill', 'and', 'would', 'be', 'as', 'narratives', 'beginnings', 'without', 'an', 'ending', 'while', 'others', 'have', 'been', 'but', 'partially', 'cleared', 'up', 'and', 'have', 'their', 'explanations', 'founded', 'rather', 'upon', 'conjecture', 'and', 'surmise', 'than', 'on', 'that', 'absolute', 'logical', 'proof', 'which', 'was', 'so', 'dear', 'to', 'him'], SparseVector(12847, {1: 1.0, 2: 1.0, 8: 1.0, 10: 1.0, 14: 1.0, 19: 1.0, 20: 1.0, 24: 1.0, 27: 1.0, 30: 1.0, 32: 1.0, 39: 1.0, 43: 1.0, 51: 1.0, 55: 1.0, 59: 1.0, 77: 1.0, 96: 1.0, 113: 1.0, 131: 1.0, 132: 1.0, 177: 1.0, 248: 1.0, 263: 1.0, 273: 1.0, 528: 1.0, 1295: 1.0, 1313: 1.0, 2087: 1.0, 2255: 1.0, 2497: 1.0, 3330: 1.0, 3811: 1.0, 3843: 1.0, 4420: 1.0, 6422: 1.0, 9481: 1.0, 10042: 1.0, 10278: 1.0, 11551: 1.0, 12606: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['i', 'signed', 'the', 'paper', 'as', 'directed', 'and', 'the', 'lawyer', 'took', 'it', 'away', 'with', 'him'], SparseVector(12847, {0: 1.0, 1: 1.0, 7: 1.0, 11: 1.0, 12: 1.0, 24: 1.0, 78: 1.0, 178: 1.0, 796: 1.0, 2090: 1.0, 3445: 1.0, 4702: 1.0}), SparseVector(12847, {9: 1.0}), 1),
 ('him', ['nigh', 'certain', 'that', 'some', 'foul', 'plot', 'had', 'been', 'woven', 'round', 'him'], SparseVector(12847, {8: 1.0, 17: 1.0, 55: 1.0, 96: 1.0, 208: 1.0, 483: 1.0, 4166: 1.0, 5266: 1.0, 6505: 1.0, 9295: 1.0}), SparseVector(12847, {9: 1.0}), 1)], schema=schema)

In [58]:
# Split the examples into train and test, use 80/20 split
df_trainset, df_testset = df_model.randomSplit((0.80, 0.20), 42)

# Print the number of training examples
print("Number training: ", df_trainset.count())

# Print the number of test examples
print("Number test: ", df_testset.count())

Number training:  31
Number test:  19


#### train model

In [59]:
# Import the logistic regression classifier
from pyspark.ml.classification import LogisticRegression

# Instantiate logistic setting elasticnet to 0.0
logistic = LogisticRegression(maxIter=10, regParam=0.4, elasticNetParam=0.0)

# Train the logistic classifer on the trainset
df_fitted = logistic.fit(df_trainset)

# Print the number of training iterations
print("Training iterations: ", df_fitted.summary.totalIterations)

Training iterations:  0


#### evaluate model

In [60]:
# Score the model on test data
testSummary = df_fitted.evaluate(df_testset)

# Print the AUC metric
print("\ntest AUC: %.3f" % testSummary.areaUnderROC)


test AUC: 1.000


In [61]:
fields = ['prediction', 'label', 'endword', 'doc', 'probability']

# Apply the model to the test data
predictions = df_fitted.transform(df_testset).select(fields)

# Print incorrect if prediction does not match label
for x in predictions.take(3):
    print()
    if x.label != int(x.prediction):
        print("INCORRECT ==> ")
    for y in fields:
        print(y,":", x[y])    

'''
INCORRECT ==> 
prediction : 0.0
label : 1
endword : him
doc : ['bind', 'him', 'bind', 'him']
probability : [0.5623411025382637,0.43765889746173625]
'''


prediction : 1.0
label : 1
endword : him
doc : ['has', 'only', 'one', 'male', 'visitor', 'but', 'a', 'good', 'deal', 'of', 'him']
probability : [0.0,1.0]

prediction : 1.0
label : 1
endword : him
doc : ['he', 'was', 'pacing', 'the', 'room', 'swiftly', 'eagerly', 'with', 'his', 'head', 'sunk', 'upon', 'his', 'chest', 'and', 'his', 'hands', 'clasped', 'behind', 'him']
probability : [0.0,1.0]

prediction : 1.0
label : 1
endword : him
doc : ['i', 'shall', 'drive', 'out', 'in', 'the', 'park', 'at', 'five', 'as', 'usual', 'she', 'said', 'as', 'she', 'left', 'him']
probability : [0.0,1.0]


"\nINCORRECT ==> \nprediction : 0.0\nlabel : 1\nendword : him\ndoc : ['bind', 'him', 'bind', 'him']\nprobability : [0.5623411025382637,0.43765889746173625]\n"

### grp